# Setup

In [1]:
run manifold_twins

In [2]:
# a = ManifoldTwinsAnalysis(idr='HICKORY', verbosity=1)
# a = ManifoldTwinsAnalysis(idr='CASCAD', verbosity=1)
a = ManifoldTwinsAnalysis(idr='BLACKSTON', verbosity=1)
# a = ManifoldTwinsAnalysis(idr='CASCAD', verbosity=1, bin_velocity=2000, max_count=100)

Loading dataset...
IDR:          BLACKSTON
Phase range: [-5.0, 5.0]
Center phase: 0.0
Bin velocity: 1000.0


100%|██████████| 415/415 [00:23<00:00, 17.45it/s]


## Configure plotting

In [3]:
%matplotlib ipympl

In [4]:
# Default settings for matplotlib figures
import matplotlib as mpl

# Choose how big to make figures. This will scale the text size.
mpl.rcParams['figure.figsize'] = (5., 4.)

# Choose the size of full-page spectra figures.
spectrum_plot_figsize = (9., 4.)

# Choose how to plot spectra. We use F_nu instead of F_lambda so that
# the features at all wavelengths can be seen. The overall scale is
# arbitrary and normalized to be ~1.
spectrum_plot_scale = a.wave**2 / 5000**2
spectrum_plot_ylabel = 'Normalized flux (erg/$cm^2$/s/Hz)'

# Choose the colormap to use for all of the plots
# from matplotlib.colors import ListedColormap
# crange = np.linspace(0, 1, 256)
# plot_cmap = ListedColormap(plt.cm.plasma(crange)[:210])
plot_cmap = plt.cm.coolwarm

# Set the DPI. This will change how big things appear in Jupyter lab
# mpl.rcParams['figure.dpi'] = 120

# Run the analysis

## Interpolate the spectra

In [5]:
# Choose how much of the model to refit. For a full analysis, use 'refit' mode.
# If you already ran everything with the exact same configuration, 'cached' will
# load the previous result. If anything changed, 'cached_uncertainty' can be used
# to keep the uncertainty model from the last fit (which takes a long time to fit,
# and is very stable) but refit everything else. Don't use cached_uncertainty for
# the final analysis, it is only for debugging!

# fit_type = 'refit'              # Refit everything
fit_type = 'cached'             # Use a full cached model that was previously fit.
# fit_type = 'cached_uncertainty' # Use only the uncertainty from a cached model.

In [6]:
print("Modeling maximum spectra with fit type '%s'" % fit_type)

if fit_type == 'refit':
    a.model_maximum_spectra(use_cache=False)
elif fit_type == 'cached':
    a.model_maximum_spectra(use_cache=True)
elif fit_type == 'cached_uncertainty':
    a.model_maximum_spectra(use_cached_model_uncertainty=True)
else:
    print("Unknown fit type! Can't handle")
    
print("Done!")

Modeling maximum spectra with fit type 'cached'
Using saved interpolation result
Done!


## Read between the lines

In [7]:
a.read_between_the_lines(fiducial_rv=2.8)

print("Done!")

Loaded cached stan model
Masking 26/203 targets whose interpolation uncertainty power is more than 0.100 of the intrinsic power.
Done!


## Do embedding

In [8]:
a.do_embedding()

## Apply standardization

In [9]:
a.fit_gp()
# a.apply_polynomial_standardization(1)

a.plot_gp()
# a.plot_gp(show_mask=True)

Fitting GP hyperparameters...
Fit result:
      fun: -60.581679263571836
 hess_inv: array([[ 0.01362433, -0.00157846,  0.00145199, -0.00682461],
       [-0.00157846,  0.00042307, -0.00039574,  0.00117954],
       [ 0.00145199, -0.00039574,  0.00097157, -0.0001324 ],
       [-0.00682461,  0.00117954, -0.0001324 ,  0.04600174]])
      jac: array([ 7.15255737e-06, -5.14984131e-05,  3.33786011e-06, -1.90734863e-06])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 151
      nit: 18
     njev: 25
   status: 2
  success: False
        x: array([-0.01267059,  0.0717383 ,  0.10495641,  2.7701184 ])
Finite difference covariance step sizes: [1.60e-04 2.00e-05 4.00e-05 2.56e-03]
Fit uncertainty: [0.072039   0.01213764 0.03502084 1.7357127 ]
Fit NMAD:        0.0795786131480306
Fit std:         0.09785320904372313


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Calculate SALT2 Hubble Residuals

In [10]:
a.calculate_salt_hubble_residuals()

Pass 0, MB=-10.430, alpha=0.146, beta=2.859
  -> new intrinsic_dispersion=0.060
Pass 1, MB=-10.430, alpha=0.146, beta=2.849
  -> new intrinsic_dispersion=0.061
Pass 2, MB=-10.430, alpha=0.146, beta=2.849
  -> new intrinsic_dispersion=0.061
Pass 3, MB=-10.430, alpha=0.146, beta=2.849
  -> new intrinsic_dispersion=0.061
Pass 4, MB=-10.430, alpha=0.146, beta=2.849
  -> new intrinsic_dispersion=0.061
SALT2 Hubble fit: 
    MB:    -10.429800911064664
    alpha: 0.14614847400720346
    beta:  2.8494438881071957
    σ_int: 0.06054515590321616
    RMS:   0.14844754353549483
    NMAD:  0.12352716751123487
    WRMS:  0.14645166479123886


## Load host galaxy data

In [11]:
a.load_host_data()

# Interpolation plots

## Examples of interpolations

In [12]:
def plot_same_night(idx, save=False):
    night_flux = a.flux[a.target_map == idx]
    phases = a.salt_phases[a.target_map == idx]
    model = a.interpolation_result['maximum_flux'][idx]
    model_err = a.interpolation_result['maximum_fluxerr'][idx]
    plt.figure()
    for flux, phase in zip(night_flux, phases):
        plt.plot(a.wave, flux * spectrum_plot_scale, label='Data (%.2f days)' % phase)
    plt.plot(a.wave, model * spectrum_plot_scale, c='k', ls='--', label='Model (0 days)')
    plt.fill_between(a.wave, (model - model_err) * spectrum_plot_scale,
                     (model + model_err) * spectrum_plot_scale, facecolor='k', alpha=0.3)
    plt.legend()
    plt.title(a.targets[idx])
    plt.ylim(0, None)
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel(spectrum_plot_ylabel)
    plt.tight_layout()
    
    if save:
        plt.savefig('./figures/interpolation_model_%s.pdf' % a.targets[idx])
    
    # plt.figure()
    # shift_frac = (a.interpolation_result['shift_fluxerr'] / a.interpolation_result['shift_flux'])[a.target_map == idx]
    # plt.plot(a.wave, shift_frac.T)
    # orig_frac = (a.fluxerr / a.flux)[a.target_map == idx]
    # plt.plot(a.wave, orig_frac.T, ls='--')
    
    
    phase_slope = a.interpolation_result['phase_slope']
    phase_quadratic = a.interpolation_result['phase_quadratic']
    gray_offsets = a.interpolation_result['gray_offsets'][a.target_map == idx]
    model_diffs = a.interpolation_result['model_diffs'][a.target_map == idx]
    
    plt.figure()
    for i, (flux, phase, gray_offset, model_diff) in enumerate(zip(night_flux, phases, gray_offsets, model_diffs)):
        plt.plot(a.wave, -2.5*np.log10(flux / model), label='Data (%.2f days)' % phase, c='C%d' % i)
    for i, (flux, phase, gray_offset, model_diff) in enumerate(zip(night_flux, phases, gray_offsets, model_diffs)):
        plt.plot(a.wave, model_diff, label='Model (%.2f days)' % phase, c='C%d' % i, ls='--')
    plt.legend(ncol=2, loc=1)
    plt.title(a.targets[idx])
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Difference from maximum light (mag)')
    plt.tight_layout()
    
    if save:
        plt.savefig('./figures/interpolation_difference_%s.pdf' % a.targets[idx])
    
    plt.figure()
    for i, (flux, phase, gray_offset, model_diff) in enumerate(zip(night_flux, phases, gray_offsets, model_diffs)):
        plt.plot(a.wave, -2.5*np.log10(flux / model) - model_diff, label='Residuals (%.2f days)' % phase, c='C%d' % i)
    plt.legend()
    plt.title(a.targets[idx])
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Interpolation residuals (mag)')
    plt.tight_layout()
    
    if save:
        plt.savefig('./figures/interpolation_residuals_%s.pdf' % a.targets[idx])
    
from ipywidgets import interact
interact(plot_same_night, idx=(0, len(a.targets)-1))

interactive(children=(IntSlider(value=101, description='idx', max=202), Checkbox(value=False, description='sav…

<function __main__.plot_same_night(idx, save=False)>

In [13]:
plot_targets = ['PTF13ayw', 'SN2004gc']
for plot_target in plot_targets:
    target_names = np.array([i.name for i in a.targets])
    plot_idx = np.where(target_names == plot_target)[0][0]

    plot_same_night(plot_idx, save=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Interpolation model

In [128]:
phase_slope = a.interpolation_result['phase_slope']
phase_quadratic = a.interpolation_result['phase_quadratic']
phase_slope_x1 = a.interpolation_result['phase_slope_x1']
phase_quadratic_x1 = a.interpolation_result['phase_quadratic_x1']

def evaluate_phase_difference(phase, x1=0):
    phase_difference = (
        phase_slope * phase
        + phase_quadratic * phase * phase
        + phase_slope_x1 * x1 * phase
        + phase_quadratic_x1 * x1 * phase * phase
    )
    
    return phase_difference

# Look at change in phase for the same x1
max_phase = a.phase_width
min_phase = -a.phase_width
num_phases = 10
phases = np.linspace(min_phase, max_phase, num_phases)

plt.figure(figsize=spectrum_plot_figsize)
# plt.figure()
norm = plt.Normalize(vmin=min_phase, vmax=max_phase)
cmap = plot_cmap
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array(phases)

for phase in phases:
    plt.plot(a.wave, evaluate_phase_difference(phase), c=cmap(norm(phase)), zorder=np.abs(phase))
    
plt.colorbar(sm, label='Phase (days)')

# plt.xlim(-5.2, 5.2)
plt.xlabel('Wavelength ($\AA$)')
plt.ylabel('Brightness relative to $t_{max,B}$ (mag)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('./figures/interpolation_phase_difference.pdf')


def plot_x1_difference(phase):
    # Look at change in phase for the same x1
    min_x1 = -2
    max_x1 = 2
    num_x1s = 10
    x1s = np.linspace(min_x1, max_x1, num_x1s)

    plt.figure(figsize=spectrum_plot_figsize)
    norm = plt.Normalize(vmin=min_x1, vmax=max_x1)
    cmap = plot_cmap
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array(x1s)

    for x1 in x1s:
        plt.plot(a.wave, evaluate_phase_difference(phase, x1) - evaluate_phase_difference(phase, 0), c=cmap(norm(x1)))

    plt.colorbar(sm, label='SALT2 $x_1$')

    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Difference relative to $x_1=0$ (mag)')
    plt.title('Difference in interpolation at %+d days' % phase)
    # plt.gca().invert_yaxis()
    plt.ylim(0.4, -0.4)
    plt.tight_layout()
    plt.savefig('./figures/interpolation_x1_difference_phase_%d.pdf' % phase)
    
for phase in [-5, -3, -1, 1, 3, 5]:
    plot_x1_difference(phase)

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:49: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:49: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:49: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:49: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Gray dispersion

In [15]:
print(a.interpolation_result['gray_dispersion_scale'])

0.026718300798931068


In [16]:
plt.figure()
plt.scatter(a.salt_phases, a.interpolation_result['gray_offsets'], s=3, label='Individual spectra')
math.plot_binned_mean(a.salt_phases, a.interpolation_result['gray_offsets'], c='C2', lw=2, label='Binned mean')
math.plot_binned_rms(a.salt_phases, a.interpolation_result['gray_offsets'], c='C3', lw=2, label='Binned RMS')
plt.xlabel('SALT2 Phase (days)')
plt.ylabel('Gray offset (mag)')
plt.legend()
plt.tight_layout()
plt.savefig('./figures/gray_offset_vs_phase.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Interpolation uncertainty

In [17]:
try:
    coefs = a.interpolation_result['phase_dispersion_coefficients']
except KeyError:
    coefs = a.stan_data['phase_dispersion_coefficients']
num_phase_coefficients = len(coefs)

def evaluate_phase_dispersion(phase):
    phase_scale = np.abs((num_phase_coefficients / 2) * (phase / a.phase_width))
    full_bins = int(np.floor(phase_scale))
    remainder = phase_scale - full_bins
    
    phase_coefficients = np.zeros(num_phase_coefficients)
    
    for j in range(full_bins + 1):
        if j == full_bins:
            weight = remainder
        else:
            weight = 1
            
        if weight == 0:
            break
            
        if phase > 0:
            phase_bin = num_phase_coefficients // 2 + j
        else:
            phase_bin = num_phase_coefficients // 2 - 1 - j
            
        phase_coefficients[phase_bin] = weight
        
    fractional_dispersion = phase_coefficients.dot(coefs)
    
    # Convert to magnitudes
    mag_dispersion = frac_to_mag(fractional_dispersion)
    
    return mag_dispersion

phases = np.linspace(-a.phase_width, a.phase_width, 1 + num_phase_coefficients)

eval_coefs = np.array([evaluate_phase_dispersion(phase) for phase in phases])

# Uncertainties for different wavelengths
plt.figure()
num_wave = 10
for i in range(num_wave):
    min_wave = a.wave[0]
    max_wave = a.wave[-1]
    wave_range = max_wave - min_wave
    target_wave = min_wave + wave_range * i / (num_wave - 1)
    idx = np.argmin(np.abs(a.wave - target_wave))
    use_wave = a.wave[idx]
    color = plt.cm.rainbow((use_wave - min_wave) / wave_range)
    plt.plot(phases, eval_coefs[:, idx], label='%d $\AA$' % use_wave, c=color)
    
plt.xlim(-5.2, 5.2)
plt.xlabel('Phase (days)')
plt.ylabel('Interpolation uncertainty (mag)')
plt.legend()
plt.tight_layout()
plt.savefig('./figures/interpolation_uncertainty_phase.pdf')

plt.figure()
for i in range(len(phases)):
    plt.plot(a.wave, eval_coefs[i], label='%.2f days' % phases[i])
plt.legend()
plt.xlabel('Wavelength $(\AA$)')
plt.ylabel('Interpolation uncertainty (mag)')
plt.tight_layout()
plt.savefig('./figures/interpolation_uncertainty_wavelength.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Model accuracy

In [18]:
max_flux = a.interpolation_result['maximum_flux']
max_fluxerr = a.interpolation_result['maximum_fluxerr']

max_magerr = frac_to_mag(max_fluxerr / max_flux)

rbtl_dispersion = frac_to_mag(a.rbtl_result['fractional_dispersion'])

def plot_uncertainties(show_rbtl=False):
    plt.figure()
    offset = 29
    
    # Make sure that we include the worst offender.
    max_loc = np.argmax(np.sum(max_magerr**2, axis=1))
    start = max_loc % offset
    
    for idx in range(start, len(a.targets), offset):
        plt.plot(a.wave, max_magerr[idx], label=a.targets[idx].name)
    plt.legend(ncol=2)
    
    plt.xlabel('Wavelength ($\AA$)')
    
    if show_rbtl:
        plt.plot(a.wave, rbtl_dispersion, label='SN intrinsic dispersion', c='k', lw=2, ls='--')
        plt.ylabel('Dispersion (mag)')
        path = './figures/interpolation_uncertainty_rbtl.pdf'
    else:
        plt.ylabel('Uncertainty on $f_{max}$ (mag)')
        path = './figures/interpolation_uncertainty_norbtl.pdf'
        
    plt.legend(ncol=2)
    plt.tight_layout()
    plt.savefig(path)
        
plot_uncertainties(False)
plot_uncertainties(True)


plt.figure()
for idx in range(len(a.targets)):
    if idx == 0:
        label = 'Individual uncertainties of $f_{max}$'
    else:
        label = ''
    plt.plot(a.wave, max_magerr[idx], label=label, alpha=0.02, c='C0')
plt.plot(a.wave, rbtl_dispersion, label='Supernova intrinsic dispersion', lw=2, ls='--', c='k')
plt.plot(a.wave, np.median(max_magerr, axis=0), label='Median uncertainty on $f_{max}$', lw=2, ls='--', c='C0')
plt.plot(a.wave, np.max(max_magerr, axis=0), label='Maximum uncertainty on $f_{max}$', c='C1')
plt.legend()
plt.ylabel('Dispersion (magnitude)')
plt.xlabel('Wavelength ($\AA$)')
plt.tight_layout()
plt.savefig('./figures/interpolation_uncertainty_median.pdf')

plt.figure()
plt.plot(a.wave, rbtl_dispersion, label='SN intrinsic dispersion', lw=2, ls='--', c='k')
plt.plot(a.wave, np.min(max_magerr, axis=0), label='Minimum $\sigma_{f,max}$')
for percentile in (25, 50, 75):
    plt.plot(a.wave, np.percentile(max_magerr, percentile, axis=0), label='%dth percentile $\sigma_{f,max}$' % percentile)
plt.plot(a.wave, np.max(max_magerr, axis=0), label='Maximum $\sigma_{f,max}$')
plt.legend(ncol=2)
plt.ylabel('Dispersion (magnitude)')
plt.xlabel('Wavelength ($\AA$)')
plt.tight_layout()
plt.savefig('./figures/interpolation_uncertainty_percentile.pdf')

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if __name__ == '__main__':


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if __name__ == '__main__':


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:38: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:54: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Contribution to the total interpolation uncertainty from various sources

In [19]:
targets = []
diffs = []
phases_1 = []
phases_2 = []
x1s = []
gray_differences = []

gray_offsets = a.interpolation_result['gray_offsets']

center_specs = a.spectra[a.center_mask]
center_gray_offsets = gray_offsets[a.center_mask]
for target_idx in range(len(a.targets)):
    near_max_spec = center_specs[target_idx]
    
    target_mask = (a.target_map == target_idx) & (~a.center_mask)
    target_specs = a.spectra[target_mask]
    target_gray_offsets = gray_offsets[target_mask]
    
    for spec_idx, target_spec in enumerate(target_specs):
        phase_diff = target_spec.phase - near_max_spec.phase
        # if np.abs(phase_diff) < 1:
            # continue

        targets.append(a.targets[target_idx])
        diff = -2.5*np.log10(target_spec.flux / near_max_spec.flux)
        diffs.append(diff)
        phases_1.append(near_max_spec.phase)
        phases_2.append(target_spec.phase)
        x1s.append(a.salt_x1[target_idx])
        
        gray_differences.append(target_gray_offsets[spec_idx] - center_gray_offsets[target_idx])

targets = np.array(targets)
diffs = np.array(diffs)
phases_1 = np.array(phases_1)
phases_2 = np.array(phases_2)
x1s = np.array(x1s)
gray_differences = np.array(gray_differences)

phase_diffs = phases_2 - phases_1

def plot_diffs(diffs, model_subtracted=False):
    sel_mask = np.zeros(len(diffs), dtype=bool)
    sel_mask[4::50] = True
    sel_mask[np.abs(phase_diffs) < 1] = False
    
    print(np.min(x1s[sel_mask]))
    print(np.max(x1s[sel_mask]))
    
    plt.figure(figsize=(8, 5))
    
    for use_idx in np.where(sel_mask)[0]:
        target = targets[use_idx]
        phase_1 = phases_1[use_idx]
        phase_2 = phases_2[use_idx]
        
        if phase_1 > phase_2:
            phase_1, phase_2 = phase_2, phase_1
        
        label = '%s, %.1f to %.1f days' % (target, phase_1, phase_2)
        
        plt.plot(a.wave, diffs[use_idx] / phase_diffs[use_idx], alpha=0.5, label=label)
        
    plt.legend(ncol=2)

    plt.ylim(-0.25, 0.25)
    plt.xlabel('Wavelength ($\AA$)')
    if model_subtracted:
        plt.ylabel('$\Delta m / \Delta t$ (data) - $\Delta m / \Delta t$ (model) (mag/day)')
    else:
        plt.ylabel('$\Delta m / \Delta t$ (data) (mag/day)')
    plt.tight_layout()

plot_diffs(diffs)
plt.savefig('./figures/raw_phase_difference.pdf')

residuals_no_x1 = []
residuals_x1 = []
for diff, phase_1, phase_2, x1 in zip(diffs, phases_1, phases_2, x1s):
    model_no_x1 = evaluate_phase_difference(phase_2, 0) - evaluate_phase_difference(phase_1, 0)
    model_x1 = evaluate_phase_difference(phase_2, x1) - evaluate_phase_difference(phase_1, x1)
    residuals_no_x1.append(diff - model_no_x1)
    residuals_x1.append(diff - model_x1)
    
residuals_no_x1 = np.array(residuals_no_x1)
residuals_x1 = np.array(residuals_x1)

residuals_gray_no_x1 = residuals_no_x1 - gray_differences[:, None]
residuals_gray_x1 = residuals_x1 - gray_differences[:, None]

plot_diffs(residuals_gray_no_x1, True)
plt.savefig('./figures/corr_phase_difference_no_x1.pdf')

plot_diffs(residuals_gray_x1, True)
plt.savefig('./figures/corr_phase_difference_x1.pdf')

def print_interpolation_residuals(min_days, max_days):
    cut = (np.abs(phase_diffs) < max_days) & (np.abs(phase_diffs) > min_days)

    def do_print(label, vals, cut):
        cut_vals = vals[cut]
        print('%20s: std=%.3f, NMAD=%.3f' % (label, math.rms(cut_vals), math.nmad(cut_vals)))

    print("Interpolation of %.1f-%.1f days:" % (min_days, max_days))
    do_print('Raw', diffs, cut)    
    do_print('Phase', residuals_no_x1, cut)    
    do_print('Phase + x1', residuals_x1, cut)    
    do_print('Phase + gray', residuals_gray_no_x1, cut)    
    do_print('Phase + x1 + gray', residuals_gray_x1, cut)    
    print("")
    
print_interpolation_residuals(0., 1.5)
print_interpolation_residuals(1.5, 2.5)
print_interpolation_residuals(2.5, 5.5)
print_interpolation_residuals(5.5, 10.5)

-1.8849701693176097
1.3058289416891107


/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

-1.8849701693176097
1.3058289416891107


/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

-1.8849701693176097
1.3058289416891107


/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Interpolation of 0.0-1.5 days:
                 Raw: std=0.052, NMAD=0.039
               Phase: std=0.045, NMAD=0.034
          Phase + x1: std=0.046, NMAD=0.034
        Phase + gray: std=0.029, NMAD=0.014
   Phase + x1 + gray: std=0.028, NMAD=0.013

Interpolation of 1.5-2.5 days:
                 Raw: std=0.115, NMAD=0.087
               Phase: std=0.080, NMAD=0.066
          Phase + x1: std=0.078, NMAD=0.064
        Phase + gray: std=0.061, NMAD=0.043
   Phase + x1 + gray: std=0.059, NMAD=0.041

Interpolation of 2.5-5.5 days:
                 Raw: std=0.178, NMAD=0.126
               Phase: std=0.094, NMAD=0.075
          Phase + x1: std=0.086, NMAD=0.070
        Phase + gray: std=0.084, NMAD=0.059
   Phase + x1 + gray: std=0.075, NMAD=0.050

Interpolation of 5.5-10.5 days:
                 Raw: std=0.272, NMAD=0.209
               Phase: std=0.127, NMAD=0.098
          Phase + x1: std=0.125, NMAD=0.106
        Phase + gray: std=0.109, NMAD=0.080
   Phase + x1 + gray: std=0.103, NMA

# Reading between the lines plots

## Show spectra before and after

In [20]:
plt.figure()
plt.plot(a.wave, a.maximum_flux[a.interp_mask][0] * spectrum_plot_scale, alpha=1, lw=1., label='Individual spectra')
plt.plot(a.wave, a.maximum_flux[a.interp_mask][1:].T * spectrum_plot_scale[:, None], lw=1.)
plt.xlabel('Wavelength ($\AA$)')
plt.ylabel(spectrum_plot_ylabel)
plt.legend()
plt.ylim(0, None)
plt.tight_layout()
plt.savefig('./figures/spectra_at_maximum.pdf')

plt.figure()
plt.plot(a.wave, a.scale_flux[a.interp_mask][0] * spectrum_plot_scale, alpha=1, lw=1., label='Individual spectra')
plt.plot(a.wave, a.scale_flux[a.interp_mask][1:].T * spectrum_plot_scale[:, None], alpha=1., lw=1.)
plt.plot(a.wave, a.mean_flux * spectrum_plot_scale, c='k', lw=2, ls='--', label='Mean spectrum')
plt.legend()
plt.xlabel('Wavelength ($\AA$)')
plt.ylabel(spectrum_plot_ylabel)
plt.ylim(0, None)
plt.tight_layout()
plt.savefig('./figures/scale_spectra.pdf')

plt.figure()
fractional_dispersion = a.rbtl_result['fractional_dispersion']
plt.plot(a.wave, a.mean_flux * spectrum_plot_scale, c='k', lw=2, ls='--', label='Mean spectrum')
plt.fill_between(a.wave, a.mean_flux * (1 - fractional_dispersion) * spectrum_plot_scale, a.mean_flux * (1 + fractional_dispersion) * spectrum_plot_scale, label='Supernova intrinsic dispersion', alpha=0.5)
plt.legend()
plt.xlabel('Wavelength ($\AA$)')
plt.ylabel(spectrum_plot_ylabel)
plt.ylim(0, None)
plt.tight_layout()
plt.savefig('./figures/scale_spectra_model.pdf')

plt.figure()
intrinsic_dispersion = frac_to_mag(a.rbtl_result['fractional_dispersion'])
plt.plot(a.wave, intrinsic_dispersion, c='k', lw=2, label='Supernova intrinsic dispersion')
plt.legend()
plt.xlabel('Wavelength ($\AA$)')
plt.ylabel('Intrinsic dispersion (mag)')
plt.ylim(0, None)
plt.tight_layout()
plt.savefig('./figures/rbtl_intrinsic_dispersion.pdf')

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  # This is added back by InteractiveShellApp.init_path()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [106]:
# Combined version

fig, axes = plt.subplots(3, 1, figsize=(spectrum_plot_figsize[0], spectrum_plot_figsize[1] * 3 - 1.5), sharex=True)

ax = axes[0]
ax.plot(a.wave, a.maximum_flux[a.interp_mask][0] * spectrum_plot_scale, alpha=1, lw=1., label='Individual spectra')
ax.plot(a.wave, a.maximum_flux[a.interp_mask][1:].T * spectrum_plot_scale[:, None], lw=1.)
ax.set_ylabel(spectrum_plot_ylabel)
ax.legend()
ax.set_title('Original spectra')
ax.set_ylim(0, None)

ax = axes[1]
ax.plot(a.wave, a.scale_flux[a.interp_mask][0] * spectrum_plot_scale, alpha=1, lw=1., label='Individual spectra')
ax.plot(a.wave, a.scale_flux[a.interp_mask][1:].T * spectrum_plot_scale[:, None], alpha=1., lw=1.)
ax.plot(a.wave, a.mean_flux * spectrum_plot_scale, c='k', lw=2, ls='--', label='Mean spectrum')
ax.legend()
ax.set_ylabel(spectrum_plot_ylabel)
ax.set_title('Dereddened spectra')
ax.set_ylim(0, None)

ax = axes[2]
intrinsic_dispersion = frac_to_mag(a.rbtl_result['fractional_dispersion'])
ax.plot(a.wave, intrinsic_dispersion, c='k', lw=2, label='Supernova intrinsic dispersion')
ax.legend()
ax.set_xlabel('Wavelength ($\AA$)')
ax.set_ylabel('Intrinsic dispersion (mag)')
ax.set_title('Recovered intrinsic dispersion')
ax.set_ylim(0, None)

plt.tight_layout()

plt.savefig('./figures/rbtl_spectra_combined.pdf')


/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Magnitudes

In [21]:
plt.figure()

plt.scatter(a.redshifts[a.interp_mask], a.mags[a.interp_mask], s=15, c='C3', label='Supernovae rejected by cuts')
plt.scatter(a.redshifts[a.good_mag_mask], a.mags[a.good_mag_mask], s=15, c='C0', label='Supernovae passing cuts')

z_range = np.linspace(0.001, 0.09, 100)
pec_vel_disp = 0.00217 / z_range
plt.fill_between(z_range, -pec_vel_disp, pec_vel_disp, alpha=0.2, label='Peculiar velocity dispersion')
plt.axvline(0.02, lw=1, ls='--', c='k', label='Redshift cutoff')

plt.xlim(0.001, 0.09)
plt.ylim(-1, 1.5)
plt.xlabel('Redshift')
plt.ylabel('RBTL measured magnitude')
plt.legend()
plt.tight_layout()
plt.savefig('./figures/rbtl_magnitude.pdf')

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
plt.figure()

plt.scatter(a.redshifts[a.good_mag_mask], a.mags[a.good_mag_mask], s=15, c='C0', label='Supernovae passing cuts')
plt.xlabel('Redshift')
plt.ylabel('RBTL measured magnitude')


z_range = np.linspace(0.001, 0.09, 100)
pec_vel_disp = 0.00217 / z_range
plt.fill_between(z_range, -pec_vel_disp, pec_vel_disp, alpha=0.2, label='Peculiar velocity dispersion')

plt.legend()
plt.tight_layout()
plt.xlim(0.01, 0.09)
plt.ylim(-0.5, 0.5)
plt.savefig('./figures/rbtl_magnitude_cut.pdf')

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
print("Raw RBTL mag std:  %.3f mag" % np.std(a.mags[a.good_mag_mask & a.interp_mask]))
print("Raw RBTL mag NMAD: %.3f mag" % math.nmad(a.mags[a.good_mag_mask & a.interp_mask]))

Raw RBTL mag std:  0.123 mag
Raw RBTL mag NMAD: 0.101 mag


In [24]:
plt.figure()

plt.scatter(a.redshifts[a.good_mag_mask], a.corr_mags[a.good_mag_mask], s=15, c='C0', label='Supernovae passing cuts')
plt.xlabel('Redshift')
plt.ylabel('RBTL measured magnitude')


z_range = np.linspace(0.001, 0.09, 100)
pec_vel_disp = 0.00217 / z_range
plt.fill_between(z_range, -pec_vel_disp, pec_vel_disp, alpha=0.2, label='Peculiar velocity dispersion')

plt.legend()
plt.tight_layout()
plt.xlim(0.01, 0.09)
plt.ylim(-0.5, 0.5)
plt.savefig('./figures/rbtl_corr_magnitude_cut.pdf')

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Manifold learning plots

## Reconstruction uncertainty

In [27]:
# Note: the total variance isn't defined for Isomap. The variances of the transfomed components
# do map onto the variance of real components though. We provide a very rough estimate of the
# measurement variance for comparison purposes... not sure how much it can be trusted...

num_show = 10

# Do an initial embedding with as many components as possible to get the full variance.
a.do_embedding(n_components=None)
variances = np.var(a.embedding[a.interp_mask], axis=0)

ref_var = np.sum(variances[:10])

plot_ref = variances[0]

print(variances[:10] / plot_ref)
print(variances[:10] / np.cumsum(variances[:10]))

plt.figure()
plt.scatter(np.arange(num_show), variances[:num_show] / plot_ref, label='Contributed variance of each component')
# plt.scatter(np.arange(num_show), variances[:num_show] / plot_ref, label='Contributed variance of each component')
# plt.axhline(0.1 * ref_var / plot_ref, label='Approximate measurement variance cut', ls='--', c='C3')
# plt.axhline(np.mean(a.interp_power_fraction[a.interp_mask]) * ref_var / plot_ref, label='Approximate mean measurement variance', ls='--', c='C2')
plt.ylim(0, None)
plt.xlabel('Component number')
plt.ylabel('Relative variance explained')
plt.xticks(np.arange(num_show), np.arange(num_show) + 1)
# plt.legend(markerfirst=False)
plt.tight_layout()

plt.savefig('./figures/isomap_component_variance.pdf')

[1.         0.60433938 0.325321   0.10772542 0.10465459 0.07846132
 0.05953441 0.05370723 0.04374868 0.03758389]
[1.         0.37669048 0.16858977 0.05287433 0.04885743 0.03533495
 0.02611117 0.02301334 0.01840119 0.0155622 ]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
np.mean(a.interp_power_fraction[a.interp_mask])# * ref_var / plot_ref

0.028232012489386313

## Twin reconstruction

In [26]:
# Plot where twins and non-twins end up for different number of components.
# We also make a summary plot.
confused_fraction = []

plot_components = np.arange(1, 6)
for n_components in plot_components:
    a.do_embedding(n_components=n_components)
    leakage_matrix = a.plot_twin_distances()
    if n_components == 1:
        title = '1 Component + Color'
    else:
        title = '%d Components + Color' % n_components
    plt.title(title)
    plt.xlabel('Recovered twinness percentile in the embedded space')
    plt.tight_layout()
    plt.savefig('./figures/twins_recovery_%d_components.pdf' % n_components)
    
    confused_fraction.append(leakage_matrix[3, 0] + leakage_matrix[3, 1])

plt.figure()
plt.scatter(np.arange(len(confused_fraction)) + 1, confused_fraction)
plt.xticks(plot_components, plot_components)
plt.ylim(0, 0.1)
plt.xlabel('Number of components (in addition to color)')
plt.ylabel('Fraction of non-twins confused as twins')
plt.tight_layout()
plt.savefig('./figures/twins_confusion.pdf')

/home/kyle/supernova/snfactory/manifold_twins/analysis/manifold_twins.py:1645: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=figsize)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,To Best 10% of twinness,To 10-20%,To 20-50%,To Worst 50% of twinness
From Best 10% of twinness,0.285623,0.283697,0.424904,0.005777
From 10-20%,0.163134,0.177264,0.560051,0.099550
From 20-50%,0.116627,0.105928,0.412369,0.365076
From Worst 50% of twinness,0.040324,0.044176,0.155644,0.759856


/home/kyle/supernova/snfactory/manifold_twins/analysis/manifold_twins.py:1645: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=figsize)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

/home/kyle/supernova/snfactory/manifold_twins/analysis/manifold_twins.py:1645: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=figsize)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,To Best 10% of twinness,To 10-20%,To 20-50%,To Worst 50% of twinness
From Best 10% of twinness,0.759307,0.209243,0.031451,0.000000
From 10-20%,0.183687,0.462428,0.353886,0.000000
From 20-50%,0.017334,0.105928,0.713032,0.163706
From Worst 50% of twinness,0.001027,0.002055,0.095159,0.901631


/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Plot slices through the manifold

In [27]:
a.do_embedding()

def plot_slice(scan_component, closest_count=10, max_dist=1., loc=np.zeros(a.embedding.shape[1] - 1)):
    loc = np.asarray(loc)
    mask = a.interp_mask

    use_embedding = a.embedding[mask]
    use_flux = a.scale_flux[mask]

    other_embedding = np.delete(use_embedding, scan_component, axis=1)
    dists = np.sqrt(np.sum((other_embedding - loc)**2, axis=1))

    dist_limit = np.min([np.sort(dists)[closest_count], max_dist])
    scan_cut = dists < dist_limit
    
    scan_embedding = use_embedding[scan_cut, scan_component]
    
    sort_embedding = np.sort(scan_embedding)
    min_comp = sort_embedding[0]
    max_comp = sort_embedding[-1]
    cmap = plot_cmap

    plt.figure(figsize=spectrum_plot_figsize)
    for spec, val in zip(use_flux[scan_cut], scan_embedding):
        plt.plot(a.wave, spec * spectrum_plot_scale, c=cmap((val - min_comp) / (max_comp - min_comp)))

    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel(spectrum_plot_ylabel)

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min_comp, vmax=max_comp))
    sm._A = []
    plt.colorbar(sm, label='Value of Component %d' % (scan_component + 1))
    plt.title('Component %d' % (scan_component + 1))
    plt.tight_layout()
    
    plt.savefig('./figures/component_%d_effect.pdf' % (scan_component + 1))

In [28]:
plot_slice(0, loc=[-0.5, -0.5])
plot_slice(1)
plot_slice(2, loc=[0, 1])

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Plot steps through component values - combined

In [138]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, axes = plt.subplots(3, 1, figsize=(spectrum_plot_figsize[0], spectrum_plot_figsize[1] * 3 - 1.5), sharex=True)

num_steps = 10

for component in range(3):
    mask = a.interp_mask

    use_embedding = a.embedding[mask, component]
    use_flux = a.scale_flux[mask]
    
    min_embedding = np.percentile(use_embedding, 5)
    max_embedding = np.percentile(use_embedding, 95)
    
    bin_edges = np.linspace(min_embedding, max_embedding, num_steps+1)
    
    bin_edges[0] = -1e20
    bin_edges[-1] = 1e20
    
    ax = axes[component]
    
    cmap = plot_cmap
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min_embedding, vmax=max_embedding))
    sm._A = []
    
    for step in range(num_steps):
        step_mask = (use_embedding >= bin_edges[step]) & (use_embedding < bin_edges[step+1])
        step_embedding = use_embedding[step_mask]

        mean_val = np.mean(step_embedding)
        step_flux = np.median(use_flux[step_mask], axis=0)
        
        # Make the extreme values of components get plotted on top if everything overlaps.
        zorder = np.abs(mean_val)
        
        ax.plot(a.wave, step_flux * spectrum_plot_scale, c=sm.to_rgba(mean_val), zorder=zorder)
        
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='2.5%', pad=0.1)
    fig.colorbar(sm, cax=cax, orientation='vertical', label='Component Value')
    # fig.colorbar(sm, cax=cax, orientation='vertical', label='Value of Component %d' % (component + 1))
    # ax.colorbar(sm, label='Value of Component %d' % (component + 1))
    ax.set_title('Component %d' % (component + 1))
    
    if component == 2:
        ax.set_xlabel('Wavelength ($\AA$)')
    
    ax.set_ylabel(spectrum_plot_ylabel)
    ax.set_ylim(0, None)
    
plt.tight_layout()

plt.savefig('./figures/component_steps_combined.pdf')

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Plot steps through component values - single

In [129]:
# a.do_embedding()

def plot_steps(component, num_steps=10, xlim=None):
    mask = a.interp_mask

    use_embedding = a.embedding[mask, component]
    use_flux = a.scale_flux[mask]
    
    min_embedding = np.percentile(use_embedding, 5)
    max_embedding = np.percentile(use_embedding, 95)
    
    bin_edges = np.linspace(min_embedding, max_embedding, num_steps+1)
    
    bin_edges[0] = -1e20
    bin_edges[-1] = 1e20
    
    plt.figure(figsize=spectrum_plot_figsize)
    
    cmap = plot_cmap
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min_embedding, vmax=max_embedding))
    sm._A = []

    if xlim is not None:
        wave_mask = (a.wave > xlim[0] - 50) & (a.wave < xlim[1] + 50)
    else:
        wave_mask = np.ones(len(a.wave), dtype=bool)
    
    for step in range(num_steps):
        step_mask = (use_embedding >= bin_edges[step]) & (use_embedding < bin_edges[step+1])
        step_embedding = use_embedding[step_mask]

        mean_val = np.mean(step_embedding)
        step_flux = np.median(use_flux[step_mask], axis=0)
        
        # if step == 0:
            # label = 'Median spectra in each component bin'
        # else:
            # label = ''
            
        # Make the extreme values of components get plotted on top if everything overlaps.
        zorder = np.abs(mean_val)
        
        # plt.plot(a.wave[wave_mask], step_flux[wave_mask], c=sm.to_rgba(mean_val), label=label)
        plt.plot(a.wave[wave_mask], step_flux[wave_mask] * spectrum_plot_scale[wave_mask], c=sm.to_rgba(mean_val), zorder=zorder)
        
    if xlim is not None:
        plt.xlim(*xlim)
        
    plt.colorbar(sm, label='Value of Component %d' % (component + 1))
    plt.title('Component %d' % (component + 1))
    
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel(spectrum_plot_ylabel)
    plt.ylim(0, None)
    
    # plt.legend()
    
    plt.tight_layout()
    
    if xlim is None:
        plt.savefig('./figures/component_%d_steps.pdf' % (component + 1))
    else:
        plt.savefig('./figures/component_%d_steps_zoom_%d_%d.pdf' % (component + 1, xlim[0], xlim[1]))

In [130]:
for component in range(3):
    plot_steps(component)
    # plot_steps(component, xlim=(3300, 4500))
    # plot_steps(component, xlim=(4900, 6700))
    # plot_steps(component, xlim=(7200, 8600))

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Comparison to original twins

In [32]:
hannah_list = np.genfromtxt('./data/fakhouri_atmax_list.txt', dtype='str')
twins_mask = np.array([i.name in hannah_list for i in a.targets])

In [33]:
a.plot_twin_pairings()
plt.xlim(0, 100)
plt.ylim(0, None)
plt.tight_layout()
plt.savefig('./figures/twin_dispersion.pdf')

RMS  20%: 0.10241635885739103
NMAD 20%: 0.09379011862452236


/home/kyle/supernova/snfactory/manifold_twins/analysis/manifold_twins.py:1708: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Twins that are poor brightness matches

In [34]:
from scipy.spatial.distance import pdist, squareform

mask = a.good_mag_mask

raw_spec_dists = pdist(a.iso_diffs[mask])
spec_dists = squareform(raw_spec_dists)
mag_diffs = squareform(pdist(a.mags[mask][:, None]))

In [35]:
dist_mask = spec_dists < np.percentile(raw_spec_dists, 20)
plt.figure()
plt.hist(mag_diffs[dist_mask])

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([463., 234., 188., 138.,  74.,  40.,  34.,  12.,   0.,   2.]),
 array([0.        , 0.05245902, 0.10491805, 0.15737707, 0.2098361 ,
        0.26229512, 0.31475415, 0.36721317, 0.4196722 , 0.47213122,
        0.52459025]),
 <a list of 10 Patch objects>)

In [36]:
idx1, idx2 = np.where(dist_mask & (mag_diffs > 0.35))
idx_mask = idx1 < idx2
idx1 = idx1[idx_mask]
idx2 = idx2[idx_mask]

def print_pairs(vals):
    for val_name, val in vals.items():
        print("%13s_1" % val_name, "%13s_2" % val_name, end='')
    print('')
        
    for i, j in zip(idx1, idx2):
        for val_name, val in vals.items():
            try:
                print('%15.3f' % val[mask][i], '%15.3f' % val[mask][j], end='')
            except TypeError:
                print('%15s' % val[mask][i], '%15s' % val[mask][j], end='')
        print('')

print_pairs({
    'target': a.targets,
    'redshift': a.redshifts,
    'color': a.colors,
    'mag': a.mags,
    'salt_mag': a.salt_hr,
    'embedding[0]': a.embedding[:, 0],
})

       target_1        target_2     redshift_1      redshift_2        color_1         color_2          mag_1           mag_2     salt_mag_1      salt_mag_2 embedding[0]_1  embedding[0]_2
   CSS110918_02 SNF20080612-003          0.038           0.033         -0.211          -0.178          0.063          -0.290         -0.000          -0.228          0.302          -0.376
       PTF11drz SNF20080612-003          0.057           0.033         -0.089          -0.178          0.092          -0.290          0.089          -0.228         -1.581          -0.376
       PTF12jqh SNF20080612-003          0.047           0.033         -0.279          -0.178          0.101          -0.290          0.078          -0.228         -1.703          -0.376
       SN2005hc        SN2007kk          0.045           0.041         -0.369          -0.237          0.234          -0.181          0.148          -0.202         -1.310          -2.487
       SN2005hc SNF20060512-002          0.045           0.051   

## Comparision to Branch classifications

In [37]:
a.do_blondin_plot()
plt.savefig('./figures/branch_classification.pdf')

/home/kyle/supernova/snfactory/manifold_twins/analysis/manifold_twins.py:737: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
a.do_component_blondin_plot()
plt.gca().get_legend().remove()
plt.savefig('./figures/branch_labels_isomap.pdf')

/home/kyle/supernova/snfactory/manifold_twins/analysis/manifold_twins.py:691: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Variation of Core Normal SNe Ia

In [131]:
indicators = a.get_indicators() 

s1 = indicators["EWSiII6355"]
s2 = indicators["EWSiII5972"]

component = 0

core_normal_cut = (s2 < 30) & (s1 > 70) & (s1 < 100) & ~np.isnan(a.embedding[:, component])

cut_flux = a.scale_flux[core_normal_cut]
cut_coord = a.embedding[core_normal_cut][:, component]

sort_flux = cut_flux[np.argsort(cut_coord)]
sort_coord = cut_coord[np.argsort(cut_coord)]

num_bins = 5

cmap = plot_cmap
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=np.percentile(cut_coord, 100 / num_bins / 2), vmax=np.percentile(cut_coord, 100 * (1 - 1. / num_bins / 2))))
sm._A = []

def plot_spec(bin_idx):
    min_idx = int(len(sort_coord) / num_bins * bin_idx)
    max_idx = int(len(sort_coord) / num_bins * (bin_idx + 1))
    bin_flux = sort_flux[min_idx:max_idx]
    
    f = np.median(bin_flux, axis=0)
    
    mean_val = np.mean(sort_coord[min_idx:max_idx])
    
    plt.plot(a.wave, f * spectrum_plot_scale, c=sm.to_rgba(mean_val), zorder=np.abs(mean_val))
    
plt.figure(figsize=spectrum_plot_figsize)
for i in range(num_bins):
    plot_spec(i)
plt.colorbar(sm, label='Value of Component %d' % (component + 1))
plt.xlabel('Wavelength ($\AA$)')
plt.ylabel(spectrum_plot_ylabel)
# plt.legend()
plt.tight_layout()
plt.savefig('./figures/core_normal_comparison.pdf')

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Recovering spectral feature indicators

In [318]:
indicators = a.get_indicators() 

In [337]:
def find_rotation(values):
    # Find the best predictor of vSi6355
    def to_min(x):
        diff = values - a.embedding.dot(x[1:]) - x[0]
        return np.sum(diff[a.interp_mask]**2)

    res = minimize(to_min, [0, 0, 0, 0])

    rotation = res.x[1:] / np.sqrt(np.sum(res.x[1:]**2))
    best_guess = a.embedding.dot(res.x[1:]) - res.x[0]
    
    return rotation, best_guess

latex_map = {
    'EWCaIIHK': 'pEW \ion{Ca}{2} HK',
    'EWSiII4000': 'pEW \ion{Si}{2} 4000\AA',
    'EWSiII5972': 'pEW \ion{Si}{2} 5972\AA',
    'EWSiII6355': 'pEW \ion{Si}{2} 6355\AA',
    'vCaIIHK': 'Velocity \ion{Ca}{2} HK',
    'vSiII6355': 'Velocity \ion{Si}{2} 6355\AA',
}

with open('./latex/indicators.tex', 'w') as f:
    for key in indicators.keys():
        if key[:3] == 'lam':
            continue
            
        rotation, best_guess = find_rotation(indicators[key])

        latex_name = latex_map[key]
        
        corrcoef = np.corrcoef(best_guess[a.interp_mask], indicators[key][a.interp_mask])[0, 1]
        
        latex_print(f, f'{latex_name} & {rotation[0]:.2f} & {rotation[1]:.2f} & {rotation[2]:.2f} & {corrcoef:.2f}')

pEW \ion{Ca}{2} HK & -0.97 & -0.19 & 0.12 & 0.88
pEW \ion{Si}{2} 4000\AA & -0.32 & 0.84 & 0.45 & 0.81
pEW \ion{Si}{2} 5972\AA & -0.04 & 0.94 & 0.33 & 0.77
pEW \ion{Si}{2} 6355\AA & -0.71 & 0.69 & 0.13 & 0.91
Velocity \ion{Ca}{2} HK & 0.75 & 0.62 & -0.21 & 0.82
Velocity \ion{Si}{2} 6355\AA & 0.51 & -0.13 & 0.85 & 0.89


In [329]:
# Find the best predictor of vSi6355
def to_min(x):
    diff = vsi - a.embedding.dot(x[1:]) - x[0]
    return np.sum(diff[a.interp_mask & ~iax_mask]**2)

res = minimize(to_min, [0, 0, 0, 0])

norm_x = res.x[1:] / np.sqrt(np.sum(res.x[1:]**2))
print(norm_x)

[-0.50757526  0.10758014 -0.85486483]


In [317]:
indicators

EWCaIIHK,EWSiII4000,EWSiII5972,EWSiII6355,lamCaIIHK,lamSiII6355,vCaIIHK,vSiII6355
float64,float64,float64,float64,float64,float64,float64,float64
110.42065298127248,7.156539304727554,4.267682719807493,74.01160754301279,3784.696759063133,6113.196759061015,-11601.497304698218,-11631.786123729733
115.88511968980427,14.244723778962888,14.519889493285104,96.11641661689501,3764.0967590631517,6136.996759060994,-13236.039231030405,-10467.671674450818
137.66437833256646,8.03400109449936,9.359226409676404,80.31669082398298,3705.396759063205,6134.596759060996,-17938.488549397873,-10584.871381198802
98.071683955134,15.053598497870988,13.040642419828158,88.41134536875856,3811.7967590631083,6129.796759061001,-9463.674454746679,-10819.398661571704
105.7656373427338,7.3190797309667435,8.914381819783797,67.7065555351691,3790.8967590631273,6114.196759061015,-11111.150325397944,-11582.789434050232
121.35964872558834,15.075244395569673,18.0559879904286,104.12015719743108,3782.696759063135,6136.996759060994,-11759.831984221417,-10467.671674450818
48.225558149127984,2.72163536091812,11.051846363465634,28.100668189730126,3805.7967590631138,6108.4967590610195,-9935.770610103174,-11862.169602243932
-8.921149217737954,-9.49216365193811,19.43997970175498,50.262872639575995,3829.9967590630918,6247.796759060893,-8035.906119549421,-5103.414104958819
124.93705931217573,10.835205363631927,11.136667985735452,70.01959942503478,3705.7967590632047,6123.796759061006,-17906.220678198326,-11112.797474030653


In [319]:
indicators = a.get_indicators() 

vsi = -indicators["vSiII6355"] / 1000

In [320]:
a.scatter(vsi, axis_1=0, axis_2=2, vmin=8, label='vSi II 6355$\AA$ ($10^3$ km/s)')
plt.savefig('./figures/line_velocities_components_13.pdf')

/home/kyle/supernova/snfactory/manifold_twins/analysis/manifold_twins.py:860: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [321]:
# Find the best predictor of vSi6355
def to_min(x):
    diff = vsi - a.embedding.dot(x[1:]) - x[0]
    return np.sum(diff[a.interp_mask & ~iax_mask]**2)

res = minimize(to_min, [0, 0, 0, 0])

norm_x = res.x[1:] / np.sqrt(np.sum(res.x[1:]**2))
print(norm_x)

pred_vsi = a.embedding.dot(res.x[1:])[a.interp_mask] + res.x[0]
mask_vsi = vsi[a.interp_mask]

min_vsi = np.min([pred_vsi, mask_vsi])
max_vsi = np.max([pred_vsi, mask_vsi])
min_vsi = min_vsi - 0.05 * (max_vsi - min_vsi)
max_vsi = max_vsi + 0.05 * (max_vsi - min_vsi)

plt.figure()
plt.scatter(pred_vsi, mask_vsi, s=20, label='Measurements')
plt.plot([min_vsi, max_vsi], [min_vsi, max_vsi], ls='--', c='k', label='One-to-one line')
plt.xlim(min_vsi, max_vsi)
plt.ylim(min_vsi, max_vsi)
plt.xlabel('Rotated Isomap vSi II 6355$\AA$ Estimate ($10^3$ km/s)')
plt.ylabel('vSi II 6355$\AA$ ($10^3$ km/s)')
plt.legend()
plt.tight_layout()
plt.savefig('./figures/isomap_vsi_recovery.pdf')

with open('./latex/vsi_commands.txt', 'w') as f:
    latex_command(f, 'vsicomp1', '$%.3f$', norm_x[0])
    latex_command(f, 'vsicomp2', '$%.3f$', norm_x[1])
    latex_command(f, 'vsicomp3', '$%.3f$', norm_x[2])
    latex_command(f, 'vsicorrcoef', '%.3f', np.corrcoef(pred_vsi, mask_vsi)[0, 1])
    latex_command(f, 'vsistd', '%.0f', np.std(pred_vsi - mask_vsi) * 1000)
    latex_command(f, 'vsinmad', '%.0f', math.nmad(pred_vsi - mask_vsi) * 1000)

[-0.50757526  0.10758014 -0.85486483]


/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

\newcommand{\vsicomp1}{$-0.508$}
\newcommand{\vsicomp2}{$0.108$}
\newcommand{\vsicomp3}{$-0.855$}
\newcommand{\vsicorrcoef}{0.894}
\newcommand{\vsistd}{523}
\newcommand{\vsinmad}{410}


In [307]:
def plot_steps(x, num_steps=10, xlim=None):
    mask = a.interp_mask

    use_embedding = x#a.embedding[mask, component]
    use_flux = a.scale_flux[mask]
    
    min_embedding = np.percentile(use_embedding, 5)
    max_embedding = np.percentile(use_embedding, 95)
    
    bin_edges = np.linspace(min_embedding, max_embedding, num_steps+1)
    
    bin_edges[0] = -1e20
    bin_edges[-1] = 1e20
    
    plt.figure(figsize=spectrum_plot_figsize)
    
    cmap = plot_cmap
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min_embedding, vmax=max_embedding))
    sm._A = []

    if xlim is not None:
        wave_mask = (a.wave > xlim[0] - 50) & (a.wave < xlim[1] + 50)
    else:
        wave_mask = np.ones(len(a.wave), dtype=bool)
    
    for step in range(num_steps):
        step_mask = (use_embedding >= bin_edges[step]) & (use_embedding < bin_edges[step+1])
        step_embedding = use_embedding[step_mask]

        mean_val = np.mean(step_embedding)
        step_flux = np.median(use_flux[step_mask], axis=0)
        
        # Make the extreme values of components get plotted on top if everything overlaps.
        # This looks bad for this plot, so don't do it
        # zorder = np.abs(mean_val - np.mean(x))
        # zorder = np.abs(mean_val)
        
        # plt.plot(a.wave[wave_mask], step_flux[wave_mask], c=sm.to_rgba(mean_val), label=label)
        plt.plot(a.wave[wave_mask], step_flux[wave_mask] * spectrum_plot_scale[wave_mask], c=sm.to_rgba(mean_val))
        
    if xlim is not None:
        plt.xlim(*xlim)
        
    plt.colorbar(sm, label='Rotated Isomap vSi II 6355$\AA$\nEstimate ($10^3$ km/s)')
    
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel(spectrum_plot_ylabel)
    plt.ylim(0, None)
    
    # plt.legend()
    
    plt.tight_layout()
    
    if xlim is None:
        plt.savefig('./figures/component_vsi_rotation_steps.pdf')
    # else:
        # plt.savefig('./figures/component_%d_steps_zoom_%d_%d.pdf' % (component + 1, xlim[0], xlim[1]))
        
plot_steps(pred_vsi)

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  from ipykernel import kernelapp as app


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Color comparison

In [308]:
a.scatter(a.colors, vmin=-0.3, vmax=0.3)

/home/kyle/supernova/snfactory/manifold_twins/analysis/manifold_twins.py:860: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
plt.figure()
plt.scatter(a.embedding[:, 1], a.colors, c=a.embedding[:, 0])

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Standardization 

## Raw magnitudes

In [42]:
a.scatter(a.mags, a.good_mag_mask, vmin=-0.2, vmax=0.2, label='Residual magnitude', invert_colorbar=True)
plt.savefig('./figures/components_12_residual_magnitude.pdf')

/home/kyle/supernova/snfactory/manifold_twins/analysis/manifold_twins.py:860: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
a.scatter(a.mags, a.good_mag_mask, vmin=-0.2, vmax=0.2, axis_1=0, axis_2=2, label='Residual magnitude', invert_colorbar=True)
plt.savefig('./figures/components_13_residual_magnitude.pdf')

/home/kyle/supernova/snfactory/manifold_twins/analysis/manifold_twins.py:860: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
a.scatter(a.mags, a.good_mag_mask, vmin=-0.2, vmax=0.2, axis_1=1, axis_2=2, label='Residual magnitude', invert_colorbar=True)
plt.savefig('./figures/components_23_residual_magnitude.pdf')

/home/kyle/supernova/snfactory/manifold_twins/analysis/manifold_twins.py:860: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## GP standardization

In [45]:
# Reset to defaults in case things got messed up
a.do_embedding()
a.fit_gp()

Fitting GP hyperparameters...
Fit result:
      fun: -60.581679263571836
 hess_inv: array([[ 0.01362433, -0.00157846,  0.00145199, -0.00682461],
       [-0.00157846,  0.00042307, -0.00039574,  0.00117954],
       [ 0.00145199, -0.00039574,  0.00097157, -0.0001324 ],
       [-0.00682461,  0.00117954, -0.0001324 ,  0.04600174]])
      jac: array([ 7.15255737e-06, -5.14984131e-05,  3.33786011e-06, -1.90734863e-06])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 151
      nit: 18
     njev: 25
   status: 2
  success: False
        x: array([-0.01267059,  0.0717383 ,  0.10495641,  2.7701184 ])
Finite difference covariance step sizes: [1.60e-04 2.00e-05 4.00e-05 2.56e-03]
Fit uncertainty: [0.072039   0.01213764 0.03502084 1.7357127 ]
Fit NMAD:        0.0795786131480306
Fit std:         0.09785320904372313


In [46]:
a.plot_gp(axis_1=0, axis_2=1)
plt.savefig('./figures/gp_mag_components_12.pdf')

a.plot_gp(axis_1=0, axis_2=2)
plt.savefig('./figures/gp_mag_components_13.pdf')

a.plot_gp(axis_1=1, axis_2=2)
plt.savefig('./figures/gp_mag_components_23.pdf')

/home/kyle/supernova/snfactory/manifold_twins/analysis/manifold_twins.py:860: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/kyle/supernova/snfactory/manifold_twins/analysis/manifold_twins.py:860: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/kyle/supernova/snfactory/manifold_twins/analysis/manifold_twins.py:860: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Check vs phases of original spectra

In [47]:
plt.figure()

# Get the mean phase for every target.
mean_phases = []
for i in range(len(a.targets)):
    mean_phases.append(np.mean(a.salt_phases[a.target_map == i]))
mean_phases = np.array(mean_phases)

# plt.scatter(a.salt_phases[a.center_mask][a.good_mag_mask], a.corr_mags[a.good_mag_mask], label='Individual observations')
# math.plot_binned_mean(a.salt_phases[a.center_mask][a.good_mag_mask], a.corr_mags[a.good_mag_mask], c='C2', label='Binned mean')
plt.scatter(mean_phases[a.good_mag_mask], a.corr_mags[a.good_mag_mask], label='Individual observations')
math.plot_binned_mean(mean_phases[a.good_mag_mask], a.corr_mags[a.good_mag_mask], c='C2', label='Binned mean')
# plt.xlabel('Phase of closest spectrum to maximum (days)')
plt.xlabel('Mean phase of observed spectra (days)')
plt.ylabel('Residual magnitude')
plt.gca().invert_yaxis()
plt.legend()

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# SALT2 comparison

In [48]:
# Load SALT2 Hubble residuals
a.calculate_salt_hubble_residuals()

Pass 0, MB=-10.430, alpha=0.146, beta=2.859
  -> new intrinsic_dispersion=0.060
Pass 1, MB=-10.430, alpha=0.146, beta=2.849
  -> new intrinsic_dispersion=0.061
Pass 2, MB=-10.430, alpha=0.146, beta=2.849
  -> new intrinsic_dispersion=0.061
Pass 3, MB=-10.430, alpha=0.146, beta=2.849
  -> new intrinsic_dispersion=0.061
Pass 4, MB=-10.430, alpha=0.146, beta=2.849
  -> new intrinsic_dispersion=0.061
SALT2 Hubble fit: 
    MB:    -10.429800911064664
    alpha: 0.14614847400720346
    beta:  2.8494438881071957
    σ_int: 0.06054515590321616
    RMS:   0.14844754353549483
    NMAD:  0.12352716751123487
    WRMS:  0.14645166479123886


## SALT2 colors

In [49]:
plt.figure()
plt.scatter(a.salt_color, a.colors, s=5)
plt.xlabel('SALT2 Color ($c$)')
plt.ylabel('RBTL Color ($A_V$)')
plt.tight_layout()

plt.savefig('./figures/salt2_color_comparison.pdf')

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## SALT2 X1

In [50]:
a.scatter(a.salt_x1, a.interp_mask, label='SALT $x_1$')
plt.savefig('./figures/salt2_x1_components.pdf')

/home/kyle/supernova/snfactory/manifold_twins/analysis/manifold_twins.py:860: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [139]:
# Find the best predictor of x1
def to_min(x):
    diff = a.salt_x1 - a.embedding.dot(x)
    return np.nanstd(diff[a.salt_mask])

res = minimize(to_min, [0, 0, 0])

norm_x = res.x / np.sqrt(np.sum(res.x**2))
print(norm_x)

plt.figure()
# plt.scatter(a.embedding.dot(res.x), a.salt_x1, c=a.salt_mask)
plt.scatter(a.embedding.dot(res.x)[~a.salt_mask], a.salt_x1[~a.salt_mask], c='C3', s=30, label='"Bad" SALT2 fits', alpha=0.8)
plt.scatter(a.embedding.dot(res.x)[a.salt_mask], a.salt_x1[a.salt_mask], c='C0', s=30, label='"Good" SALT2 fits', alpha=0.8)
plt.plot([-3, 3], [-3, 3], ls='--', c='k', label='One-to-one line')
plt.xlim(-3, 3)
plt.ylim(-3, 3)
plt.xlabel('Rotated Isomap components')
plt.ylabel('SALT2 $x_1$')
plt.legend()
plt.savefig('./figures/rotated_isomap_salt_x1.pdf')


print(np.corrcoef(a.embedding.dot(res.x)[a.interp_mask & a.salt_mask], a.salt_x1[a.interp_mask & a.salt_mask]))

[ 0.21070948 -0.96283285 -0.16898052]


/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  # This is added back by InteractiveShellApp.init_path()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[1.         0.81948099]
 [0.81948099 1.        ]]


## SALT2 outliers (Type Iax)

In [268]:
iax_mask = (a.embedding[:, 0] > 4.) & (a.embedding[:, 1] < -3)
a.targets[iax_mask]

array([Target(name="SN2005cc"), Target(name="SN2011ay")], dtype=object)

In [53]:
# Outlier spectra
mask = (a.embedding[:, 0] > 4) & (a.embedding[:, 1] < -2)
print(a.targets[mask])
print(a.colors[mask])
print(a.redshifts[mask])
print(a.mags[mask])

# Ref spectrum
ref_target = 'SNF20070803-005'
for idx2, target in enumerate(a.targets):
    if target.name == ref_target:
        break
        
# idx2 = np.where((a.embedding[:, 0] > 4.8) & (a.embedding[:, 1] < 2))[0][0]

plt.figure()
for i in np.where(mask)[0]:
    plt.plot(a.wave, a.scale_flux[i] * spectrum_plot_scale, label=a.targets[i].name)
    
plt.plot(a.wave, a.scale_flux[idx2] * spectrum_plot_scale, c='k', ls='--', label=a.targets[idx2].name)
print(a.embedding[idx2])

plt.legend()

plt.xlabel('Wavelength ($\AA$)')
plt.ylabel(spectrum_plot_ylabel)
plt.tight_layout()

plt.savefig('./figures/type_iax_comparison.pdf')

[Target(name="SN2005cc") Target(name="SN2011ay")]
[1.15559577 1.03167635]
[0.00785178 0.02125125]
[ 1.43131201 -0.49525916]


/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  app.launch_new_instance()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[ 5.16103866 -1.19708043 -0.7113846 ]


## Other outliers

In [140]:
a.targets[(a.embedding[:, 1] > 4.)]

array([Target(name="PTF11kjn"), Target(name="SN2009hs"),
       Target(name="SNNGC6430")], dtype=object)

In [146]:
a.scatter(a.embedding[:, 2])

/home/kyle/supernova/snfactory/manifold_twins/analysis/manifold_twins.py:860: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [152]:
a.scatter(a.colors, axis_1=1, axis_2=2)

/home/kyle/supernova/snfactory/manifold_twins/analysis/manifold_twins.py:860: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [144]:
# Outlier spectra
mask = (a.embedding[:, 0] < -4)
print(a.targets[mask])
print(a.colors[mask])
print(a.redshifts[mask])
print(a.mags[mask])

# Ref spectrum
# ref_target = 'SNF20070803-005'
# for idx2, target in enumerate(a.targets):
    # if target.name == ref_target:
        # break
        
# idx2 = np.where((a.embedding[:, 0] > 4.8) & (a.embedding[:, 1] < 2))[0][0]

plt.figure()
for i in np.where(mask)[0]:
    plt.plot(a.wave, a.scale_flux[i] * spectrum_plot_scale, label=a.targets[i].name)
    
# plt.plot(a.wave, a.scale_flux[idx2] * spectrum_plot_scale, c='k', ls='--', label=a.targets[idx2].name)
print(a.embedding[idx2])

plt.legend()

plt.xlabel('Wavelength ($\AA$)')
plt.ylabel(spectrum_plot_ylabel)
plt.tight_layout()

plt.savefig('./figures/type_iax_comparison.pdf')

[Target(name="LSQ13aiz") Target(name="PTF10nlg") Target(name="PTF10ygu")
 Target(name="PTF12grk") Target(name="PTF12iiq") Target(name="SN2007qe")
 Target(name="SN2010kg") Target(name="SNF20071021-000")
 Target(name="SNUGC11797")]
[ 0.00975344  0.29436231  0.73000614  0.02246374  0.0328221  -0.0238338
  0.21688556  0.0577835  -0.02575588]
[0.01029347 0.0558428  0.02604325 0.06936815 0.02841461 0.02280403
 0.01618157 0.02639854 0.0176469 ]
[-0.67022278         nan         nan -0.00209661  0.09290853  0.09613255
         nan  0.05467956 -0.88115352]


/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  app.launch_new_instance()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[ 5.16103866 -1.19708043 -0.7113846 ]


## SALT2 magnitudes vs components

In [54]:
a.scatter(a.salt_hr, mask=a.good_salt_mask & a.interp_mask, vmin=-0.3, vmax=0.3, label='SALT2-corrected residual magnitude', invert_colorbar=True)
# a.scatter(a.salt_hr, mask=a.good_salt_mask & a.interp_mask, vmin=-0.3, vmax=0.3, label='SALT2 Hubble residuals', invert_colorbar=True)
plt.savefig('./figures/salt2_hr_components.pdf')

/home/kyle/supernova/snfactory/manifold_twins/analysis/manifold_twins.py:860: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [55]:
plt.figure()

use_x = a.embedding[:, 0]

mask = a.salt_mask & a.redshift_color_mask & a.interp_mask
plt.scatter(use_x[mask], a.salt_hr[mask], label='Individual supernovae')
math.plot_binned_mean(use_x[mask], a.salt_hr[mask], c='C2', lw=2, label='Binned mean')
plt.legend()

plt.gca().invert_yaxis()

plt.xlabel('Component 1')
plt.ylabel('SALT2-corrected residual magnitude')

plt.tight_layout()
plt.savefig('./figures/salt2_hr_component_1.pdf')

plt.figure()
plt.hist(a.salt_hr[(use_x < 2) & mask], 10, (-0.6, 0.4), alpha=0.3, color='C0', label='Component 1 < 2', density=True)
plt.hist(a.salt_hr[(use_x < 2) & mask], 10, (-0.6, 0.4), histtype='step', lw=2, color='C0', density=True)
plt.hist(a.salt_hr[(use_x > 2) & mask], 10, (-0.6, 0.4), alpha=0.3, color='C1', label='Component 1 > 2', density=True)
plt.hist(a.salt_hr[(use_x > 2) & mask], 10, (-0.6, 0.4), histtype='step', lw=2, color='C1', density=True)

plt.gca().invert_xaxis()

plt.xlabel('SALT2 residual magnitude')
plt.ylabel('Normalized counts')
plt.legend()

plt.tight_layout()
plt.savefig('./figures/salt2_hr_hist.pdf')

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## SALT2 + Isomap standardization

In [56]:
a.fit_gp(kind='salt_raw')
a.plot_gp(kind='salt_raw', vmin=-0.5, vmax=0.5)

plt.savefig('./figures/salt_gp_component_12.pdf')

Fitting GP hyperparameters...
Fit result:
      fun: -84.54178986772008
 hess_inv: array([[ 1.83420612e-02, -2.44134640e-04, -1.66486099e-03,
         1.00531615e-03],
       [-2.44134640e-04,  6.31372864e-05,  8.11372426e-05,
        -1.46236085e-04],
       [-1.66486099e-03,  8.11372426e-05,  4.14684419e-03,
        -2.43232210e-03],
       [ 1.00531615e-03, -1.46236085e-04, -2.43232210e-03,
         2.56787638e-03]])
      jac: array([ 2.86102295e-06, -1.04904175e-05,  2.95639038e-05,  3.05175781e-05])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 996
      nit: 44
     njev: 164
   status: 2
  success: False
        x: array([ 2.83209164,  0.09115336, -0.45166565,  9.19249767])
Finite difference covariance step sizes: [3.20e-04 2.00e-05 1.60e-04 2.56e-03]
Fit uncertainty: [0.14575785 0.0087175  0.27255561 5.33591983]
Fit NMAD:        0.10645415401213752
Fit std:         0.11639661580869723


/home/kyle/supernova/snfactory/manifold_twins/analysis/manifold_twins.py:860: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
plt.figure()
m = a.good_salt_mask & a.interp_mask
plt.scatter(a.salt_x1[m], (a.corr_mags - a.salt_hr)[m], c=a.embedding[m, 0], cmap=plt.cm.coolwarm, vmin=-3, vmax=3)
plt.gca().invert_yaxis()
plt.colorbar(label='Value of Isomap component 1')
plt.xlabel('SALT2 $x_1$')
plt.ylabel('Difference between SALT2 and\nSALT2 + Isomap standardization (mag)')
plt.tight_layout()
plt.savefig('./figures/salt_isomap_difference.pdf')

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
# Redo the original GP fit to put the analysis back in the original state
a.fit_gp(verbose=True)

Fitting GP hyperparameters...
Fit result:
      fun: -60.581679263571836
 hess_inv: array([[ 0.01362433, -0.00157846,  0.00145199, -0.00682461],
       [-0.00157846,  0.00042307, -0.00039574,  0.00117954],
       [ 0.00145199, -0.00039574,  0.00097157, -0.0001324 ],
       [-0.00682461,  0.00117954, -0.0001324 ,  0.04600174]])
      jac: array([ 7.15255737e-06, -5.14984131e-05,  3.33786011e-06, -1.90734863e-06])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 151
      nit: 18
     njev: 25
   status: 2
  success: False
        x: array([-0.01267059,  0.0717383 ,  0.10495641,  2.7701184 ])
Finite difference covariance step sizes: [1.60e-04 2.00e-05 4.00e-05 2.56e-03]
Fit uncertainty: [0.072039   0.01213764 0.03502084 1.7357127 ]
Fit NMAD:        0.0795786131480306
Fit std:         0.09785320904372313


In [59]:
target = 'BOSS38'

for j, i in enumerate(a.targets):
    if target in i.name:
        idx = j
        break
else:
    raise Exception(f"No target {target} found!")
    
print(f"target:        {a.targets[idx].name}")
print(f"isomap coord:  {a.embedding[idx]}")
print(f"SALT x1:       {a.salt_x1[idx]}")

gp_pred = a.predict_gp(a.embedding[idx], a.salt_color[idx], kind='salt_raw')[0]
# salt_pred = a.salt_alpha * a.salt_x1[idx] - a.salt_beta * a.salt_color[idx]
salt_pred = a.salt_hr_raw[idx] - a.salt_hr[idx]

print(f"GP prediction: {gp_pred}")
print(f"SALT2 pred:    {salt_pred}")
print(f"pred diff:     {salt_pred - gp_pred}")

# print(f"HR difference: {(a.corr_mags - a.salt_hr)[idx]}")

target:        SNBOSS38
isomap coord:  [ 2.51422933  2.18692745 -1.81835255]
SALT x1:       1.4419367231774203
GP prediction: -0.1438799524496813
SALT2 pred:    0.11566306284643671
pred diff:     0.259543015296118


## Same SALT2 parameters but not at the same location in the manifold

In [60]:
def outer_diff(x):
    return (x - x[:, None])

count = np.array([len(i.spectra) for i in a.targets])
first_phase = np.array([i.spectra[0].phase for i in a.targets])
mask = (count > 8) & (a.redshifts > 0.03) & (first_phase < -2)

salt_x1_diff = outer_diff(a.salt_x1)
salt_c_diff = outer_diff(a.salt_color)
salt_diff = outer_diff(a.salt_x1)**2 + 100 * outer_diff(a.salt_color)**2
manif_diff = outer_diff(a.embedding[:, 0])**2 + outer_diff(a.embedding[:, 1])**2 + outer_diff(a.embedding[:, 2])**2

phase_diff = outer_diff(a.salt_phases[a.center_mask])

np.fill_diagonal(salt_diff, np.nan)
np.fill_diagonal(manif_diff, np.nan)

salt_diff[~mask] = np.nan
salt_diff[:, ~mask] = np.nan
manif_diff[~mask] = np.nan
manif_diff[:, ~mask] = np.nan

In [61]:
cut = (salt_x1_diff > 0) & (salt_x1_diff < 0.2) & (np.abs(salt_c_diff) < 0.02) & (manif_diff > 30) & (np.abs(phase_diff) < 1.0)
print("Num objects:  %s" % np.sum(cut))
salt_diff_idx = [i[2] for i in np.where(cut)]

t1 = a.targets[salt_diff_idx[0]]
t2 = a.targets[salt_diff_idx[1]]

print("target 1:     %s" % t1)
print("target 2:     %s" % t2)
print("redshifts:    %.4f, %.4f" % (a.redshifts[salt_diff_idx[0]], a.redshifts[salt_diff_idx[1]]))
print("x_1 values:   %+.3f, %+.3f" % (a.salt_x1[salt_diff_idx[0]], a.salt_x1[salt_diff_idx[1]]))
print("c values:     %+.3f, %+.3f" % (a.salt_color[salt_diff_idx[0]], a.salt_color[salt_diff_idx[1]]))
print("mags:         %.3f, %.3f" % (a.mags[salt_diff_idx[0]], a.mags[salt_diff_idx[1]]))
print("salt HRs:     %.3f, %.3f" % (a.salt_hr[salt_diff_idx[0]], a.salt_hr[salt_diff_idx[1]]))
print("corr mags:    %.3f, %.3f" % (a.corr_mags[salt_diff_idx[0]], a.corr_mags[salt_diff_idx[1]]))
print("embedding 1:  %s" % a.embedding[salt_diff_idx[0]])
print("embedding 2:  %s" % a.embedding[salt_diff_idx[1]])

salt_comparision_mb_diff = np.abs(
    a.salt_hr[salt_diff_idx[0]]
    - a.salt_hr[salt_diff_idx[1]]
)
dm_1 = frac_to_mag(t1.salt_fit['x0_err'] / t1.salt_fit['x0'])
dm_2 = frac_to_mag(t2.salt_fit['x0_err'] / t2.salt_fit['x0'])

salt_comparison_mb_diff_err = np.sqrt(dm_1**2 + dm_2**2)
print("mag_diff:     %.3f +/- %.3f" % (salt_comparision_mb_diff, salt_comparison_mb_diff_err))

Num objects:  3
target 1:     SNIC3573
target 2:     PTF11mkx
redshifts:    0.0670, 0.0536
x_1 values:   +0.558, +0.729
c values:     +0.047, +0.035
mags:         -0.195, nan
salt HRs:     -0.201, -0.511
corr mags:    -0.153, nan
embedding 1:  [-0.14072518 -1.44239144  2.94944758]
embedding 2:  [ 5.09677022 -1.15869851 -0.51807872]
mag_diff:     0.310 +/- 0.059


/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in greater
  """Entry point for launching an IPython kernel.


In [62]:
bands = ['U', 'B', 'V', 'R', 'I']

band_colors = {
    'U': 'C4',
    'B': 'C0',
    'V': 'C2',
    'R': 'C3',
    'I': 'C1',
}

gap = 1.

def plot_lightcurve(target, marker='o', ls='--', mfc='none', label_bands=True):
    photometry = target.get_photometry(bands, clip_filter=True)
    
    # Cut late phases
    photometry = photometry[photometry['time'] < 30]
        
    ref_mag = target.salt_fit['fitted_model'].source_peakmag('snfb', 'ab')
    
    for offset, band in enumerate(bands):
        band_photometry = photometry[photometry['band'] == 'snf%s' % band.lower()]
        
        if band == 'B':
            label = target.name
        else:
            label = ''
            
        color = band_colors[band]
        
        phases = band_photometry['time']
        mags = -2.5*np.log10(band_photometry['flux']) - ref_mag - (offset - 1) * gap
        magerrs = band_photometry['magerr']
    
        plt.errorbar(phases, mags, magerrs, fmt='none', c=color)
        plt.plot(phases, mags, label=label, marker=marker, c=color, mfc=mfc, ls=ls)
        
        if label_bands:
            plt.text(phases[-1] + 2, mags[-1] + 0.2, '%s%+d' % (band, offset - 1), c=color)
            plt.xlim(None, phases[-1] + 9)
            
def plot_salt_lightcurve(target, phase_min=-6, phase_max=50):
    model = target.salt_fit['fitted_model']
    phases = np.linspace(phase_min, phase_max, 200)
    
    # Compute the reference magnitude for B-band at max.
    ref_mag = model.source_peakmag('snfb', 'ab')

    # Plot each light curve
    for offset, band in enumerate(bands):
        mags = model.bandmag('snf%s' % band.lower(), 'ab', phases) - ref_mag - (offset - 1) * gap
        plt.plot(phases, mags, c='k')


plt.figure(figsize=(5, 5))
plot_lightcurve(t1, marker='^', ls='', mfc=None, label_bands=False)
plot_lightcurve(t2, marker='o', ls='', mfc='none', label_bands=True)
# plot_salt_lightcurve(t1)
plt.legend()
plt.xlabel('Phase (days)')
plt.ylabel('Normalized magnitude + offset')
plt.ylim(4.8, -2.5)
plt.tight_layout()

plt.savefig('./figures/same_salt_comparison.pdf')

# plt.gca().invert_yaxis()

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:55: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
plt.figure()

def plot_spec(idx):
    spec = a.spectra[a.center_mask][idx]

    ref_mag = spec.target.salt_fit['fitted_model'].source_peakmag('snfb', 'ab')
    
    label = '%s, %.2f days' % (spec.target.name, spec.phase)
    
    scale = 10**(+0.4*ref_mag)
    
    plt.plot(a.wave, spec.flux * scale * spectrum_plot_scale, label=label)

plot_spec(salt_diff_idx[0])
plot_spec(salt_diff_idx[1])

plt.legend()

plt.xlabel('Restframe wavelength ($\AA$)')
plt.ylabel(spectrum_plot_ylabel)
plt.tight_layout()

plt.savefig('./figures/same_salt_spectra.pdf')

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [64]:
# Dump details to latex
with open('latex/salt_comparison.tex', 'w') as f:
    latex_print(f, "")
    t1 = a.targets[salt_diff_idx[0]]
    t2 = a.targets[salt_diff_idx[1]]
    latex_command(f, 'saltcompnamea', '%s', t1.name)
    latex_command(f, 'saltcompnameb', '%s', t2.name)
    latex_command(f, 'saltcompxonea', '%.3f $\\pm$ %.3f', (t1['salt2.X1'], t1['salt2.X1.err']))
    latex_command(f, 'saltcompxoneb', '%.3f $\\pm$ %.3f', (t2['salt2.X1'], t2['salt2.X1.err']))
    latex_command(f, 'saltcompca', '%.3f $\\pm$ %.3f', (t1['salt2.Color'], t1['salt2.Color.err']))
    latex_command(f, 'saltcompcb', '%.3f $\\pm$ %.3f', (t2['salt2.Color'], t2['salt2.Color.err']))
    latex_command(f, 'saltcompcoorda', '%.2f', a.embedding[salt_diff_idx[0], 0])
    latex_command(f, 'saltcompcoordb', '%.2f', a.embedding[salt_diff_idx[1], 0])
    latex_command(f, 'saltcompmagbdiff', '%.3f $\\pm$ %.3f', (salt_comparision_mb_diff, salt_comparison_mb_diff_err))


\newcommand{\saltcompnamea}{SNIC3573}
\newcommand{\saltcompnameb}{PTF11mkx}
\newcommand{\saltcompxonea}{0.443 $\pm$ 0.171}
\newcommand{\saltcompxoneb}{0.558 $\pm$ 0.132}
\newcommand{\saltcompca}{0.025 $\pm$ 0.028}
\newcommand{\saltcompcb}{0.042 $\pm$ 0.027}
\newcommand{\saltcompcoorda}{-0.14}
\newcommand{\saltcompcoordb}{5.10}
\newcommand{\saltcompmagbdiff}{0.310 $\pm$ 0.059}


# Host galaxy correlations

In [65]:
a.fit_gp(kind='rbtl', verbose=False)
rbtl_isomap_mags = a.corr_mags
a.fit_gp(kind='salt_raw', verbose=False)
salt_isomap_mags = a.corr_mags

In [66]:
def calc_step(sample, sides, errs, axis=None):
    vals_1 = np.average(sample, weights=sides / errs**2, axis=axis)
    vals_2 = np.average(sample, weights=(1-sides) / errs**2, axis=axis)
    
    return vals_1, vals_2, vals_2 - vals_1

def bootstrap_step(sample, sides, errs=1):
    return math.bootstrap_statistic(calc_step, sample, sides, errs)

def likelihood_step(residuals, prob, errs=1.):
    def calc_likelihood(x):
        s1, s2, err1, err2 = x
        
        var1 = errs**2 + err1**2
        var2 = errs**2 + err2**2

        likelihood = np.sum(-np.log(
            prob * 1 / np.sqrt(2 * np.pi * var1) * np.exp(-(residuals - s1)**2 / var1)
            + (1 - prob) * 1 / np.sqrt(2 * np.pi * var2) * np.exp(-(residuals - s2)**2 / var2)
        ))
        
        return likelihood
    
    res = minimize(calc_likelihood, [0., 0., 0.1, 0.1], method='BFGS')
    param_errs = np.sqrt(np.diag(res.hess_inv))

    # Estimate the variances with the intrinsic components.
    total_vars = errs**2 + prob * res.x[2]**2 + (1 - prob) * res.x[3]**2

    step_means = (res.x[0], res.x[1], res.x[1] - res.x[0])
    step_errs = (param_errs[0], param_errs[1], np.sqrt(param_errs[0]**2 + param_errs[1]**2))
    
    return step_means, step_errs, total_vars
    

def int_disp(vals, pec_vel_disps, axis=None):
    std = np.std(vals, ddof=1, axis=axis)
    corr = np.mean(pec_vel_disps**2, axis=axis)
    corr_std = np.sqrt(np.clip(std**2 - corr, 0, None))

    return corr_std

def analyze_host_variable(variable, mags, mask, uncertainties=None, threshold=None, use_probability=True, plot=True, y_label='Residual magnitudes'):
    use_mask = mask & a.host_mask
    host_data = a.host_data[use_mask]
    use_mags = mags[use_mask]
            
    # Default thresholds from Rigault et al. 2019
    if threshold is None:
        if variable == 'lssfr':
            threshold = -10.8
        elif variable == 'gmass':
            threshold = 10
        else:
            raise Exception("Must specify threshold!")
            
    use_var = host_data[variable]
    use_var_low = host_data[variable + '_low']
    use_var_high = host_data[variable + '_up']
    
    # Figure out labels.
    if variable == 'lssfr':
        x_label = 'log(lsSFR)'
    elif variable == 'gmass':
        x_label = 'log($M_* / M_\odot$) (global)'
            
    # Figure out which weights to use for the step. We want to actually use
    # the probabilities if they are available rather than hard cuts as is done
    # in Rigault et al. 2018.
    label = variable
    use_weights = None

    if use_probability:
        if variable == 'lssfr':
            plot_color = host_data['p_young']
            use_weights = 1 - host_data['p_young'] / 100
            label = '$P_{Young}$'
        elif variable == 'gmass':
            plot_color = host_data['p_highmass']
            use_weights = 1 - host_data['p_highmass'] / 100
            label = '$P_{high\ mass}$'

    if use_weights is None:
        # Backup: do hard cuts.
        use_weights = use_var < threshold
        plot_color = use_weights

    if uncertainties is None:
        # If we don't have explicit uncertainties, just use the peculiar velocity contributions.
        uncertainties = a.get_peculiar_velocity_uncertainty()

    use_uncertainties = uncertainties[use_mask]
    
    step_means, step_errs, total_var = likelihood_step(use_mags, use_weights, use_uncertainties)

    if plot:
        print("Step size: %.3f ± %.3f mag" % (step_means[2], step_errs[2]))
        plt.figure()
        plt.errorbar(use_var, use_mags, xerr=(-use_var_low, use_var_high), yerr=np.sqrt(total_var), fmt='.', c='gray', alpha=0.5, zorder=-2)
        plt.scatter(use_var, use_mags, s=100, c=plot_color, edgecolors='gray', cmap=plt.cm.viridis_r)
        
        # Threshold
        plt.axvline(threshold, c='k', lw=2, ls='--')
        
        # Show means of each side
        plot_min, plot_max = plt.xlim()
        mean_low, mean_high, mean_diff = step_means
        mean_low_err, mean_high_err, mean_diff_err = step_errs
        plt.plot([plot_min, threshold], [mean_low, mean_low], c='k', zorder=-1)
        plt.fill_between([plot_min, threshold], [mean_low - mean_low_err, mean_low - mean_low_err], [mean_low + mean_low_err, mean_low + mean_low_err], color=plt.cm.viridis(1000), alpha=0.5, zorder=-3)
        plt.plot([threshold, plot_max], [mean_high, mean_high], c='k', zorder=-1)
        plt.fill_between([threshold, plot_max], [mean_high - mean_high_err, mean_high - mean_high_err], [mean_high + mean_high_err, mean_high + mean_high_err], color=plt.cm.viridis(0), alpha=0.5, zorder=-3)

        plt.xlabel(x_label)
        plt.ylabel(y_label)

        plt.xlim(plot_min, plot_max)
        plt.ylim(-0.6, 0.6)


        plt.colorbar(label=label)
        plt.tight_layout()

    return step_means[-1], step_errs[-1]

## lsSFR plots

In [67]:
analyze_host_variable('lssfr', a.salt_hr, a.good_salt_mask, a.salt_hr_raw_uncertainties, y_label='SALT2 + $x_1$ residual magnitudes')
plt.savefig('./figures/lssfr_salt_x1.pdf')

Step size: 0.113 ± 0.023 mag


/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:98: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [68]:
analyze_host_variable('lssfr', a.salt_hr, a.good_salt_mask & a.good_mag_mask, a.salt_hr_raw_uncertainties, y_label='SALT2 + $x_1$ residual magnitudes')
plt.savefig('./figures/lssfr_salt_x1_cuts.pdf')
analyze_host_variable('lssfr', rbtl_isomap_mags, a.good_salt_mask & a.good_mag_mask, y_label='RBTL + Isomap residual magnitudes')
plt.savefig('./figures/lssfr_rbtl_isomap_cuts.pdf')
analyze_host_variable('lssfr', salt_isomap_mags, a.good_salt_mask & a.good_mag_mask, a.salt_hr_raw_uncertainties, y_label='SALT2 + Isomap residual magnitudes')
plt.savefig('./figures/lssfr_salt_isomap_cuts.pdf')

Step size: 0.084 ± 0.034 mag


/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:98: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Step size: 0.047 ± 0.027 mag


/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:98: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Step size: 0.026 ± 0.032 mag


/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:98: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [69]:
a.fit_gp()

Fitting GP hyperparameters...
Fit result:
      fun: -60.581679263571836
 hess_inv: array([[ 0.01362433, -0.00157846,  0.00145199, -0.00682461],
       [-0.00157846,  0.00042307, -0.00039574,  0.00117954],
       [ 0.00145199, -0.00039574,  0.00097157, -0.0001324 ],
       [-0.00682461,  0.00117954, -0.0001324 ,  0.04600174]])
      jac: array([ 7.15255737e-06, -5.14984131e-05,  3.33786011e-06, -1.90734863e-06])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 151
      nit: 18
     njev: 25
   status: 2
  success: False
        x: array([-0.01267059,  0.0717383 ,  0.10495641,  2.7701184 ])
Finite difference covariance step sizes: [1.60e-04 2.00e-05 4.00e-05 2.56e-03]
Fit uncertainty: [0.072039   0.01213764 0.03502084 1.7357127 ]
Fit NMAD:        0.0795786131480306
Fit std:         0.09785320904372313


## Global mass plots

In [70]:
analyze_host_variable('gmass', a.salt_hr, a.good_salt_mask, a.salt_hr_raw_uncertainties, y_label='SALT2 + $x_1$ residual magnitudes')
plt.savefig('./figures/gmass_salt_x1.pdf')

Step size: -0.101 ± 0.023 mag


/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:98: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [71]:
analyze_host_variable('gmass', a.salt_hr, a.good_salt_mask & a.good_mag_mask, y_label='SALT2 + $x_1$ residual magnitudes')
plt.savefig('./figures/gmass_salt_x1_cuts.pdf')
analyze_host_variable('gmass', rbtl_isomap_mags, a.good_salt_mask & a.good_mag_mask, y_label='RBTL + Isomap residual magnitudes')
plt.savefig('./figures/gmass_rbtl_isomap_cuts.pdf')
analyze_host_variable('gmass', salt_isomap_mags, a.good_salt_mask & a.good_mag_mask, y_label='SALT2 + Isomap residual magnitudes')
plt.savefig('./figures/gmass_salt_isomap_cuts.pdf')

Step size: -0.072 ± 0.033 mag


/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:98: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Step size: -0.029 ± 0.025 mag


/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:98: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Step size: 0.001 ± 0.035 mag


/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:98: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Summary plots

In [72]:
analyze_host_variable('lssfr', salt_isomap_mags, a.good_mag_mask & a.interp_mask & a.good_salt_mask, plot=False)

(0.030259571552653887, 0.03229067567104885)

In [73]:
analyze_host_variable('lssfr', salt_isomap_mags, a.interp_mask & a.good_salt_mask, plot=False)

(0.03233525257036619, 0.024623614070543205)

In [74]:
host_results = {
    # 'Local Mass':  [0.008,  0.04,   -0.02,  0.028],
    # 'Local SFR':   [0.013,  -0.007, -0.018, 0.015],
    # 'Local SSFR':  [0.058,  0.0956, 0.029,  0.070],
    # 'Global Mass': [0.0258,  0.061, -0.0136,-0.008],
    'Local Mass':  [-0.0041,  0.0488],
    'Local SSFR':  [0.0164,  0.0744],
    'Global Mass': [-0.0121,  0.0517],
}

err = 0.025

plt.figure()
labels = []
for idx, (key, values) in enumerate(host_results.items()):
    for val_idx, value in enumerate(values):
        if val_idx >= 2:
            continue
        marker = 'oovv'[val_idx]
        # color = 'C%d' % (val_idx % 2)
        color = 'C%d' % (val_idx % 2)
        if idx == 0:
            label = ['Manifold twins', 'SALT2', 'Manifold twins color cut', 'SALT2 color cut'][val_idx]
        else:
            label = None
        gap = 0.1
        # xpos = idx - 3.*gap + 2*gap * val_idx + gap * (val_idx // 2)
        # xpos = idx - 1.5*gap + gap * val_idx
        xpos = idx - 0.5*gap + gap * val_idx
        plt.errorbar(xpos, value, err, c=color, alpha=1.)
        plt.plot(xpos, value, marker=marker, c=color, label=label)
        
    labels.append(key)

plt.xticks(np.arange(len(labels)), labels=labels)
plt.axhline(0., c='k')
plt.xlim(-0.5, 2.5)
# plt.ylim(-0.05, 0.13)

plt.ylabel('Step size (mag)')
plt.legend(loc=2)

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  del sys.path[0]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [75]:
plt.savefig('host_correlations.eps')

In [76]:
host_results = {
    # 'Local Mass':  [0.008,  0.04,   -0.02,  0.028],
    # 'Local SFR':   [0.013,  -0.007, -0.018, 0.015],
    # 'Local SSFR':  [0.058,  0.0956, 0.029,  0.070],
    # 'Global Mass': [0.0258,  0.061, -0.0136,-0.008],
    'Local Mass':  [0.008,  0.04,   -0.004, 0.051],
    'Local SFR':   [0.013,  -0.007, 0.0047, -0.031],
    'Local SSFR':  [0.058,  0.0956, 0.027, 0.065],
    'Global Mass': [0.0258,  0.061, -0.001, 0.040],
}

err = 0.025

plt.figure()
labels = []
for idx, (key, values) in enumerate(host_results.items()):
    for val_idx, value in enumerate(values):
        if val_idx >= 2:
            continue
        marker = 'oovv'[val_idx]
        # color = 'C%d' % (val_idx % 2)
        color = 'C%d' % (val_idx % 2)
        if idx == 0:
            label = ['Manifold twins', 'SALT2', 'Manifold twins color cut', 'SALT2 color cut'][val_idx]
        else:
            label = None
        gap = 0.1
        # xpos = idx - 3.*gap + 2*gap * val_idx + gap * (val_idx // 2)
        # xpos = idx - 1.5*gap + gap * val_idx
        xpos = idx - 0.5*gap + gap * val_idx
        plt.errorbar(xpos, value, err, c=color, alpha=0.3)
        plt.plot(xpos, value, marker=marker, c=color, label=label)
        
    labels.append(key)

plt.xticks(np.arange(len(labels)), labels=labels)
plt.axhline(0., c='k')
# plt.ylim(-0.05, 0.13)

plt.ylabel('Step size (mag)')
plt.legend(loc=2)

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Attrition and LaTeX variables for things

## Attrition

In [112]:
with open('./latex/attrition_parametrization.tex', 'w') as f:
    latex_print(f, "\\textbf{General selection requirements} & \\\\")
    latex_print(f, "Initial sample                                        & %d \\\\" % len(a.dataset.targets))
    latex_print(f, "More than 5 spectra                                   & %d \\\\" % a.attrition_enough_spectra)
    latex_print(f, "SALT2 date of maximum light uncertainty < 1 day       & %d \\\\" % a.attrition_salt_daymax)
    latex_print(f, "At least one spectrum within 5 days of maximum light  & %d \\\\" % a.attrition_range)
    latex_print(f, "At least one spectrum with S/N 3300-3800~\AA\ > 100   & %d \\\\" % a.attrition_usable)
    latex_print(f, "\hline")
    latex_print(f, "\\textbf{Manifold learning selection requirements} & \\\\")
    latex_print(f, r"\textbf{(Section~\ref{sec:isomap_sample})} & \\")
    latex_print(f, "Spectrum at max. uncertainty < 10\\%% of intrinsic power        & %d \\\\" % np.sum(a.interp_mask))

\textbf{General selection requirements} & \\
Initial sample                                        & 415 \\
More than 5 spectra                                   & 280 \\
SALT2 date of maximum light uncertainty < 1 day       & 272 \\
At least one spectrum within 5 days of maximum light  & 251 \\
At least one spectrum with S/N 3300-3800~\AA\ > 100   & 203 \\
\hline
\textbf{Manifold learning selection requirements} & \\
\textbf{(Section~\ref{sec:isomap_sample})} & \\
Spectrum at max. uncertainty < 10\% of intrinsic power        & 177 \\


In [78]:
with open('./latex/attrition.tex', 'w') as f:
    latex_print(f, "\\textbf{General selection requirements} & \\\\")
    latex_print(f, "Initial sample                                        & %d \\\\" % len(a.dataset.targets))
    latex_print(f, "More than 5 spectra                                   & %d \\\\" % a.attrition_enough_spectra)
    latex_print(f, "SALT2 date of maximum light uncertainty < 1 day       & %d \\\\" % a.attrition_salt_daymax)
    latex_print(f, "At least one spectrum within 5 days of maximum light  & %d \\\\" % a.attrition_range)
    latex_print(f, "At least one spectrum with S/N 3300-3800~\AA\ > 100   & %d \\\\" % a.attrition_usable)
    latex_print(f, "\hline")
    latex_print(f, "\\textbf{Estimation of the spectra at maximum light} & \\\\")
    latex_print(f, r"\textbf{(Section~\ref{sec:int_disp_uncertainty})} & \\")
    latex_print(f, "Spectrum uncertainty < 10\\%% of intrinsic power        & %d \\\\" % np.sum(a.interp_mask))
    latex_print(f, "\hline")
    latex_print(f, "\\textbf{Valid supernova brightness requirements} & \\\\")
    latex_print(f, r"\textbf{(Section~\ref{sec:magnitude_requirements})} & \\")
    latex_print(f, "Host galaxy redshift available                        & %d \\\\" % np.sum(a.interp_mask & (a.redshift_errs < 0.004)))
    latex_print(f, "Host galaxy redshift above 0.02                       & %d \\\\" % np.sum(a.interp_mask & (a.redshift_errs < 0.004) & (a.redshifts > 0.02)))
    latex_print(f, "Measured $A_V$ < 0.5 mag                              & %d \\\\" % np.sum(a.interp_mask & a.redshift_color_mask))
    latex_print(f, "Blinded training subsample                            & %d \\\\" % np.sum(a.good_mag_mask))
    latex_print(f, "\hline")
    latex_print(f, "\\textbf{Comparisons to SALT2 fits} & \\\\")
    latex_print(f, r"\textbf{(Section~\ref{sec:salt2_standardization})} & \\")
    latex_print(f, "SNfactory SALT2 selection requirements                & %d \\\\"% np.sum(a.salt_mask))
    latex_print(f, "Passes host galaxy redshift and color requirements    & %d \\\\"% np.sum(a.good_salt_mask))
    latex_print(f, "Has a valid interpolation to maximum light            & %d \\\\"% np.sum(a.good_salt_mask & a.interp_mask))
    latex_print(f, "Blinded training subsample                            & %d \\\\"% np.sum(a.good_salt_mask & a.good_mag_mask))

\textbf{General selection requirements} & \\
Initial sample                                        & 415 \\
More than 5 spectra                                   & 280 \\
SALT2 date of maximum light uncertainty < 1 day       & 272 \\
At least one spectrum within 5 days of maximum light  & 251 \\
At least one spectrum with S/N 3300-3800~\AA\ > 100   & 203 \\
\hline
\textbf{Estimation of the spectra at maximum light} & \\
\textbf{(Section~\ref{sec:int_disp_uncertainty})} & \\
Spectrum uncertainty < 10\% of intrinsic power        & 177 \\
\hline
\textbf{Valid supernova brightness requirements} & \\
\textbf{(Section~\ref{sec:magnitude_requirements})} & \\
Host galaxy redshift available                        & 172 \\
Host galaxy redshift above 0.02                       & 148 \\
Measured $A_V$ < 0.5 mag                              & 138 \\
Blinded training subsample                            & 75 \\
\hline
\textbf{Comparisons to SALT2 fits} & \\
\textbf{(Section~\ref{sec:salt2_standardiz

## General variables

In [79]:
# Define a bunch of functions to make things easier.
def latex_host_step(file, name, var, mags, mask):
    step, step_err = analyze_host_variable(var, mags, mask, plot=False)
    latex_command(file, name, '%.3f $\\pm$ %.3f', (np.abs(step), step_err))

a.fit_gp(kind='salt_raw', verbose=False)
salt_isomap_mags = a.corr_mags.copy()

a.fit_gp(verbose=False)
rbtl_isomap_mags = a.corr_mags.copy()

with open('latex/commands.tex', 'w') as f:
    latex_print(f, "")
    latex_command(f, 'numdatasetsne', '%d', len(a.dataset.targets))
    latex_command(f, 'numdatasetspectra', '%d', np.sum([len(i.spectra) for i in a.dataset.targets]))
    latex_print(f, "")
    latex_command(f, 'nummanifoldsne', '%d', len(a.targets))
    latex_command(f, 'nummanifoldspectra', '%d', len(a.spectra))
    latex_command(f, 'numinterpsne', '%d', np.sum(a.interp_mask))
    latex_print(f, "")
    latex_command(f, 'numsnftrain', '%d', np.sum([i.subset == 'training' for i in a.targets[a.interp_mask]]))
    latex_command(f, 'numsnfvalid', '%d', np.sum([i.subset == 'validation' for i in a.targets[a.interp_mask]]))
    latex_command(f, 'numsnfother', '%d', np.sum([i.subset not in ['training', 'validation'] for i in a.targets[a.interp_mask]]))
    latex_print(f, "")
    latex_command(f, 'numsnredshift', '%d', np.sum(a.interp_mask & (a.redshift_errs >= 0.004)))
    latex_command(f, 'numlowredshift', '%d', np.sum(a.interp_mask & (a.redshifts <= 0.02)))
    latex_command(f, 'numhighav', '%d', np.sum(a.interp_mask & (a.colors - np.nanmedian(a.colors) >= 0.5)))
    latex_print(f, "")
    latex_command(f, 'nummagsne', '%d', np.sum(a.interp_mask & a.redshift_color_mask))
    latex_command(f, 'nummagsnetrain', '%d', np.sum(a.good_mag_mask))
    latex_command(f, 'nummagsnevalidation', '%d', np.sum(a.interp_mask & a.redshift_color_mask & ~a.good_mag_mask))
    latex_print(f, "")
    latex_command(f, 'saltparammb', '%.2f', a.salt_MB)
    latex_command(f, 'saltparamalpha', '%.3f', a.salt_alpha)
    latex_command(f, 'saltparambeta', '%.2f', a.salt_beta)
    latex_command(f, 'saltparamsigmaint', '%.3f', a.salt_intrinsic_dispersion)
    # latex_command(f, 'saltparamrms', '%.3f', np.std(a.salt_hr[a.good_salt_mask]))
    latex_std(f, 'saltparamrms', a.salt_hr[a.good_salt_mask])
    latex_nmad(f, 'saltparamnmad', a.salt_hr[a.good_salt_mask])
    latex_command(f, 'saltparamwrms', '%.3f', a.salt_wrms)
    latex_command(f, 'saltparammindisp', '%.2f', np.min(a.salt_hr_uncertainties[a.good_salt_mask]))
    latex_command(f, 'saltparammaxdisp', '%.2f', np.max(a.salt_hr_uncertainties[a.good_salt_mask]))
    latex_print(f, "")
    latex_std(f, 'rawrbtlmagstd', a.mags[a.good_mag_mask])
    latex_nmad(f, 'rawrbtlmagnmad', a.mags[a.good_mag_mask])
    latex_print(f, "")
    latex_command(f, 'twinrbtlmagstd', '%.3f', a.twins_rms)
    latex_command(f, 'twinrbtlmagnmad', '%.3f', a.twins_nmad)
    latex_print(f, "")
    latex_std(f, 'saltcomprawrbtlmagstd', a.mags[a.good_mag_mask & a.good_salt_mask])
    latex_std(f, 'saltcompsaltmagstd', a.salt_hr[a.good_mag_mask & a.good_salt_mask])

    a.fit_gp(verbose=False, kind='salt_raw')
    gp_uncertainties = np.sqrt(np.diag(a.gp_hyperparameter_covariance))
    latex_print(f, "")
    latex_command(f, 'saltgpcolor', '%.2f $\\pm$ %.2f', (a.gp_hyperparameters[0], gp_uncertainties[0]))
    latex_command(f, 'saltgpintdisp', '%.3f $\\pm$ %.3f', (a.gp_hyperparameters[1], gp_uncertainties[1]))
    latex_command(f, 'saltgpkernelamp', '%.3f $\\pm$ %.3f', (np.abs(a.gp_hyperparameters[2]), gp_uncertainties[2]))
    latex_command(f, 'saltgpkernellengthscale', '%.2f $\\pm$ %.2f', (a.gp_hyperparameters[3], gp_uncertainties[3]))
    latex_std(f, 'saltgprms', a.corr_mags[a.good_salt_mask & a.interp_mask])
    latex_std(f, 'saltgpcompsaltrms', a.salt_hr[a.good_salt_mask & a.interp_mask])

    a.fit_gp(verbose=False)
    gp_uncertainties = np.sqrt(np.diag(a.gp_hyperparameter_covariance))
    latex_print(f, "")
    latex_command(f, 'rbtlgpcolor', '%.2f $\\pm$ %.2f', (a.fiducial_rv * (1 + a.gp_hyperparameters[0]), a.fiducial_rv * gp_uncertainties[0]))
    latex_command(f, 'rbtlgpintdisp', '%.3f $\\pm$ %.3f', (a.gp_hyperparameters[1], gp_uncertainties[1]))
    latex_command(f, 'rbtlgpkernelamp', '%.3f $\\pm$ %.3f', (np.abs(a.gp_hyperparameters[2]), gp_uncertainties[2]))
    latex_command(f, 'rbtlgpkernellengthscale', '%.2f $\\pm$ %.2f', (a.gp_hyperparameters[3], gp_uncertainties[3]))

    latex_print(f, "")
    latex_std(f, 'rbtlgprms', a.corr_mags[a.good_mag_mask])
    latex_command(f, 'rbtlgpnmad', '%.3f', math.nmad(a.corr_mags[a.good_mag_mask]))

    latex_print(f, "")
    x1 = a.salt_hr[(a.embedding[:, 0] < 2) & a.good_salt_mask & a.interp_mask]
    x2 = a.salt_hr[(a.embedding[:, 0] > 2) & a.good_salt_mask & a.interp_mask]
    m1 = np.mean(x1)
    m2 = np.mean(x2)
    err1 = np.std(x1) / np.sqrt(len(x1))
    err2 = np.std(x2) / np.sqrt(len(x2))
    latex_command(f, 'saltisomapdiff', '%.3f $\\pm$ %.3f', (np.abs(m1-m2), np.sqrt(err1**2 + err2**2)))

    latex_print(f, "")
    latex_command(f, 'pecvelcontribution', '%.3f', np.sqrt(np.mean(a.get_peculiar_velocity_uncertainty()[a.good_mag_mask & a.good_salt_mask]**2)))

    latex_print(f, "")
    latex_host_step(f, 'lssfrsaltxifull', 'lssfr', a.salt_hr, a.good_salt_mask)
    latex_host_step(f, 'gmasssaltxifull', 'gmass', a.salt_hr, a.good_salt_mask)
    latex_host_step(f, 'lssfrsaltxicut', 'lssfr', a.salt_hr, a.good_salt_mask & a.good_mag_mask)
    latex_host_step(f, 'gmasssaltxicut', 'gmass', a.salt_hr, a.good_salt_mask & a.good_mag_mask)
    latex_host_step(f, 'lssfrsaltisomapcut', 'lssfr', salt_isomap_mags, a.good_salt_mask & a.good_mag_mask)
    latex_host_step(f, 'gmasssaltisomapcut', 'gmass', salt_isomap_mags, a.good_salt_mask & a.good_mag_mask)
    latex_host_step(f, 'lssfrrbtlisomapcut', 'lssfr', a.corr_mags, a.good_salt_mask & a.good_mag_mask)
    latex_host_step(f, 'gmassrbtlisomapcut', 'gmass', a.corr_mags, a.good_salt_mask & a.good_mag_mask)
    latex_command(f, 'hostcutsnsnetrain', '%d', np.sum(a.good_mag_mask & a.good_salt_mask & a.host_mask))
    latex_command(f, 'hostcutsnsnefull', '%d', np.sum(a.redshift_color_mask & a.interp_mask & a.good_salt_mask & a.host_mask))


\newcommand{\numdatasetsne}{415}
\newcommand{\numdatasetspectra}{3945}

\newcommand{\nummanifoldsne}{203}
\newcommand{\nummanifoldspectra}{598}
\newcommand{\numinterpsne}{177}

\newcommand{\numsnftrain}{80}
\newcommand{\numsnfvalid}{78}
\newcommand{\numsnfother}{19}

\newcommand{\numsnredshift}{5}
\newcommand{\numlowredshift}{24}
\newcommand{\numhighav}{18}

\newcommand{\nummagsne}{138}
\newcommand{\nummagsnetrain}{75}
\newcommand{\nummagsnevalidation}{63}

\newcommand{\saltparammb}{-10.43}
\newcommand{\saltparamalpha}{0.146}
\newcommand{\saltparambeta}{2.85}
\newcommand{\saltparamsigmaint}{0.061}
\newcommand{\saltparamrms}{0.149 $\pm$ 0.011}
\newcommand{\saltparamnmad}{0.124 $\pm$ 0.014}
\newcommand{\saltparamwrms}{0.146}
\newcommand{\saltparammindisp}{0.13}
\newcommand{\saltparammaxdisp}{0.22}

\newcommand{\rawrbtlmagstd}{0.123 $\pm$ 0.012}
\newcommand{\rawrbtlmagnmad}{0.101 $\pm$ 0.017}

\newcommand{\twinrbtlmagstd}{0.102}
\newcommand{\twinrbtlmagnmad}{0.094}

\newcommand{\saltcomp

## Standardization comparison

In [80]:
def int_disp(vals, pec_vel_disps, axis=None):
    std = np.std(vals, ddof=1, axis=axis)
    corr = np.mean(pec_vel_disps**2, axis=axis)
    corr_std = np.sqrt(np.clip(std**2 - corr, 0, None))

    return corr_std

def get_stat_str(all_mags, cut, function, *args):
    line_str = ""
    for mags in all_mags:
        if line_str:
            line_str += " &"
            
        use_mags = mags[cut]
        if np.any(np.isnan(use_mags)):
            line_str += "%20s" % "--"
        else:
            res, res_err = math.bootstrap_statistic(function, use_mags, *args)
            line_str += "%7.3f $\pm$ %6.3f" % (res, res_err)
    
    line_str += " \\\\"
    
    return line_str

def make_table(f, all_mags, cut, label):
    stats = {
        'NMAD': (math.nmad,),
        'Standard deviation': (np.std,),
        'Pec. vel. removed': (int_disp, a.get_peculiar_velocity_uncertainty()[cut]),
    }
    for i, (stat_name, stat_args) in enumerate(stats.items()):
        if len(label) > i:
            prefix = label[i]
        else:
            prefix = ""
            
        if i == 0:
            num_sne = "%d" % np.sum(cut)
        else:
            num_sne = ""
            
        stat_str = get_stat_str(all_mags, cut, *stat_args)
        latex_print(f, "%20s & %5s & %20s & %s" % (prefix, num_sne, stat_name, stat_str))
        
# RBTL only
rbtl_mags = a.mags.copy()
rbtl_mags[~a.good_mag_mask] = np.nan
        
# RBTL + Isomap
a.fit_gp(verbose=False)
rbtl_isomap_mags = a.corr_mags.copy()
rbtl_isomap_mags[~a.good_mag_mask] = np.nan

# SALT2
salt_mags = a.salt_hr.copy()
salt_mags[~a.good_salt_mask] = np.nan

# SALT2 + Isomap
a.fit_gp(kind='salt_raw', verbose=False)
salt_isomap_mags = a.corr_mags.copy()
salt_isomap_mags[~a.good_salt_mask] = np.nan
salt_isomap_mags[~a.interp_mask] = np.nan

all_mags = [rbtl_mags, rbtl_isomap_mags, salt_mags, salt_isomap_mags]

with open('./latex/dispersions.tex', 'w') as f:
    # RBTL only
    make_table(f, all_mags, a.interp_mask & a.redshift_color_mask & a.train_mask, ['Maximum spectrum', '+ training cuts'])
    latex_print(f, "\hline")

    # SALT2 only
    make_table(f, all_mags, a.good_salt_mask, ['SALT2 fit cuts'])
    latex_print(f, "\hline")

    # SALT2 + Isomap
    make_table(f, all_mags, a.good_salt_mask & a.interp_mask, ['Maximum spectrum', '+ SALT2 fit cuts'])
    latex_print(f, "\hline")

    # All
    make_table(f, all_mags, a.interp_mask & a.redshift_color_mask & a.salt_mask & a.train_mask, ['All cuts'])

    Maximum spectrum &    75 &                 NMAD &   0.101 $\pm$  0.017 &  0.080 $\pm$  0.012 &                  -- &                  -- \\
     + training cuts &       &   Standard deviation &   0.123 $\pm$  0.012 &  0.098 $\pm$  0.010 &                  -- &                  -- \\
                     &       &    Pec. vel. removed &   0.109 $\pm$  0.014 &  0.080 $\pm$  0.013 &                  -- &                  -- \\
\hline
      SALT2 fit cuts &   152 &                 NMAD &                   -- &                  -- &  0.124 $\pm$  0.014 &                  -- \\
                     &       &   Standard deviation &                   -- &                  -- &  0.148 $\pm$  0.011 &                  -- \\
                     &       &    Pec. vel. removed &                   -- &                  -- &  0.139 $\pm$  0.011 &                  -- \\
\hline
    Maximum spectrum &   131 &                 NMAD &                   -- &                  -- &  0.114 $\pm$  0.014 &  

## Host galaxy steps

In [81]:
# All of these functions are defined in the Host galaxy correlations section. Make
# sure to run that first.
def print_row(f, mags, cut, label, cut_label):
    step_vars = [
        ('lssfr', '$\\Delta_{lsSFR}$', +1),
        ('gmass', '$\\Delta_{mass}$', -1),
    ]
    stat_str = "%20s & %20s" % (label, cut_label)
    for i, (var_name, var_label, var_sign) in enumerate(step_vars):
        var, var_err = analyze_host_variable(var_name, mags, cut, plot=False)
        stat_str += " & %.3f $\pm$ %.3f" % (var_sign * var, var_err)
        
    stat_str += " \\\\"
    latex_print(f, stat_str)
    
with open('./latex/host_steps.tex', 'w') as f:
    print_row(f, a.salt_hr, a.good_salt_mask, 'SALT2 + $x_1$', 'SALT2')
    print_row(f, a.salt_hr, a.good_salt_mask & a.good_mag_mask, 'SALT2 + $x_1$', 'SALT2 + max + train')
    print_row(f, rbtl_isomap_mags, a.good_salt_mask & a.good_mag_mask, 'RBTL + Isomap', 'SALT2 + max + train')
    print_row(f, salt_isomap_mags, a.good_salt_mask & a.good_mag_mask, 'SALT2 + Isomap', 'SALT2 + max + train')

       SALT2 + $x_1$ &                SALT2 & 0.118 $\pm$ 0.024 & 0.107 $\pm$ 0.024 \\
       SALT2 + $x_1$ &  SALT2 + max + train & 0.095 $\pm$ 0.032 & 0.072 $\pm$ 0.033 \\
       RBTL + Isomap &  SALT2 + max + train & 0.047 $\pm$ 0.027 & 0.029 $\pm$ 0.025 \\
      SALT2 + Isomap &  SALT2 + max + train & 0.030 $\pm$ 0.032 & -0.001 $\pm$ 0.035 \\


# Dump parameters

In [82]:
import pandas as pd

In [83]:
pd.DataFrame({
    'name': [i.name for i in a.targets],
    't1': a.embedding[:, 0],
    't2': a.embedding[:, 1],
    't3': a.embedding[:, 2],
    'salt_x1': a.salt_x1,
    'salt_mag': a.salt_hr,
}).to_csv('./manifold_parameters.csv')